In [1]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../../'))

from config.config_managers import DashboardConfigManager
from dataManager import DataManager
from dash import Dash
import pandas as pd
import plotly.express as px
from abc import ABC, abstractmethod
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2025-09-21 19:09:37 - INFO - PyTorch version 2.2.2 available.


In [2]:
# CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/analysis-config.yaml")
CONFIG_PATH = Path("/Users/ahmed/Desktop/Dashboard/analysis-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)
dash_data = data_manager.load_data()

2025-09-21 19:09:39 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ahmed/Desktop/Dashboard/deformar-dashboard/notebooks/analysis/My Drive
2025-09-21 19:09:39 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2025-09-21 19:09:39 - INFO - Loading Dashboard Data from  /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/18 [00:00<?, ?it/s]

2025-09-21 19:09:42 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ahmed/Desktop/Dashboard/deformar-dashboard/notebooks/analysis/My Drive
2025-09-21 19:09:42 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2025-09-21 19:09:42 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ahmed/Desktop/Dashboard/deformar-dashboard/notebooks/analysis/My Drive
2025-09-21 19:09:42 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2025-09-21 19:09:43 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ahmed/Desktop/Dashboard/deformar-dashboard/notebooks/analysis/My Drive
2025-09-21 19:09:43 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.youn

  0%|          | 0/18 [00:00<?, ?it/s]

2025-09-21 19:09:52 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ahmed/Desktop/Dashboard/deformar-dashboard/notebooks/analysis/My Drive
2025-09-21 19:09:52 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2025-09-21 19:09:52 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ahmed/Desktop/Dashboard/deformar-dashboard/notebooks/analysis/My Drive
2025-09-21 19:09:52 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com


In [3]:
dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data.columns

Index(['Sentence Ids', 'Token Positions', 'Words', 'Tokens', 'Word Pieces',
       'Core Tokens', 'True Labels', 'Token Selector Id', 'Pred Labels',
       'Agreements', 'X', 'Y', 'Labels', 'Loss Values', 'Token Ids',
       'Global Id', 'True Silhouette', 'Pred Silhouette', 'K=3',
       'Boundary Clusters', 'K=4', 'Entity Clusters', 'K=9', 'Token Clusters',
       'Consistency Count', 'Inconsistency Count', 'Total Train Occurrences',
       'Local Token Entropy', 'Token Max Entropy', 'Dataset Token Entropy',
       'Local Word Entropy', 'Word Max Entropy', 'Dataset Word Entropy',
       'Tokenization Rate', 'Error Type', 'O Confidence', 'B-PER Confidence',
       'I-PER Confidence', 'B-ORG Confidence', 'I-ORG Confidence',
       'B-LOC Confidence', 'I-LOC Confidence', 'B-MISC Confidence',
       'I-MISC Confidence', 'Prediction Entropy', 'Prediction Max Entropy',
       'Token Confidence', 'Variability', 'Pre X', 'Pre Y',
       'Strict True Entities', 'Strict Pred Entities', 'True E

In [4]:
dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data['Vocabulary Status'].value_counts()

Vocabulary Status
IV     22340
OOV     7371
Name: count, dtype: int64

In [5]:
from typing import Dict, Iterator, Iterable, Tuple, List, Optional
import pandas as pd
import numpy as np

from seqeval.metrics.sequence_labeling import get_entities
from seqeval.scheme import Entities, auto_detect
from collections import Counter, defaultdict


_SPLIT_LABEL    = {"train": "Train", "test": "Test", "validation": "Validation", "dev": "Validation"}
_DATASET_LABEL  = {"ANERCorp_CamelLab": "ANERCorp", "conll2003": "CoNLL-2003"}


# pretty/variant names -> raw corpus keys (extend as needed)
_DATASET_MAP    = {
    "ANERCorp": "ANERCorp_CamelLab",
    "CoNLL-2003": "conll2003",
    "ANERCorp_CamelLab_arabertv02": "ANERCorp_CamelLab",
    "conll2003_bert": "conll2003",
}

LANGUAGE_MAP = {
    "ANERCorp_CamelLab": "Arabic",
    "conll2003": "English",
    "ANERCorp_CamelLab_arabertv02": "Arabic",
    "conll2003_bert": "English",
}


_TAG_NORMALIZE  = {"B-PERS": "B-PER", "I-PERS": "I-PER"}

_DEFAULT_TAGS = ['B-LOC','I-LOC','B-PER','I-PER','B-ORG','I-ORG','B-MISC','I-MISC']

BASE_DATASETS = {"ANERCorp_CamelLab", "conll2003"}


_DEFAULT_ENTITY_SPANS = ["LOC", "PER", "ORG", "MISC"]
_ENTITY_NORMALIZE = {"PERS": "PER"}  # unify ANER's PERS→PER

class BaseDatasetHelper:
    """Only cross-cutting utilities. No analysis-specific logic here."""

    def __init__(self, corpora: Dict):
        self.corpora = corpora

    # ---------- variant → dataset keys ----------
    def resolve_language_keys(self, variant: str) -> List[str]:
        """Return raw corpus keys for a given variant or 'combined'."""
        if variant == "combined":
            # choose the ones you want to include in 'combined'
            # return [k for k in self.corpora.keys() if k in ("ANERCorp_CamelLab", "conll2003")]
            return [k for k in self.corpora.keys() if k in BASE_DATASETS]
        # pretty/alias → raw
        if variant in _DATASET_MAP:
            return [_DATASET_MAP[variant]]
        # # assume it's already a raw key
        return [variant]

    # ---------- labels ----------
    def lang_label(self, ds_key: str) -> str:
        return LANGUAGE_MAP.get(ds_key, ds_key)

    def split_label(self, split_key: str) -> str:
        return _SPLIT_LABEL.get(split_key.lower(), split_key.title())

    # ---------- iterate splits ----------
    def iter_splits(self, ds_key: str) -> Iterator[Tuple[str, str, pd.DataFrame]]:
        """
        Yields (split_key, split_label, df) for a dataset key.
        - Prefers train/test if present
        - Drops validation for conll2003 (customize rules here)
        """
        splits = self.corpora[ds_key]["splits"]
        order = [k for k in ("train", "test") if k in splits] or list(splits.keys())
        if ds_key == "conll2003":
            order = [k for k in order if k.lower() != "validation"]
        for sk in order:
            yield sk, self.split_label(sk), splits[sk]

    # ---------- tag normalization (optional) ----------
    def normalize_tag_column(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Ensure a 'Tag' column exists and normalize label variants.
        Accepts 'True Labels' or 'Tag' as source.
        """
        tag_col = "True Labels" if "True Labels" in df.columns else ("Tag" if "Tag" in df.columns else None)
        if tag_col is None:
            raise ValueError("Expected a tag column: 'True Labels' or 'Tag'.")
        out = df.copy()
        out[tag_col] = out[tag_col].replace(_TAG_NORMALIZE)
        return out.rename(columns={tag_col: "Tag"})
    
    @staticmethod
    def _normalize_tag(tag: str) -> str:
        return _TAG_NORMALIZE.get(tag, tag)




class BaseDashDataProcessor:
    def __init__(self, dash_data: Dict[str, "DashboardData"]):
        self.dash_data = dash_data  # raw objects with .train_data / .analysis_data
        self.corpora = {}           # filled by build_corpora()

    # labels (optional pretty names)
    # def ds_label(self, variant_key: str) -> str:
    #     """
    #     Example:
    #       variant_key='ANERCorp_CamelLab_arabertv02'
    #       DATA_MAP[variant_key] -> 'ANERCorp_CamelLab'
    #       _DATASET_LABEL['ANERCorp_CamelLab'] -> 'ANERCorp'
    #     Falls back gracefully if mappings are missing.
    #     """
    #     ds_key = _DATASET_MAP.get(variant_key, variant_key)
    #     return _DATASET_LABEL.get(ds_key, ds_key)
    def ds_label(self, variant_key: str) -> str:
        return LANGUAGE_MAP.get(variant_key, variant_key)


    def normalise_data(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df[df['Labels'] != -100].copy()
        if 'True Labels' in df.columns:
            df['True Labels'] = df['True Labels'].replace({'B-PERS': 'B-PER', 'I-PERS': 'I-PER'})
        return df

    def _resolve_keys_for_variant(self, variant: str) -> list[str]:
        """Map a variant to concrete keys in dash_data."""
        
        if variant == "combined":
            return list(self.dash_data.keys())
        return [variant]

    def build_corpora(self, variant: str) -> dict:
        """
        selected_variant:
          - '<variant_name>' e.g. 'ANERCorp_CamelLab_arabertv02'
          - 'combined'       -> include all variants present in dash_data

        Populate self.corpora for the requested variant.
        Output shape:
          self.corpora[<Dataset Label>]['splits'] = {'Train': df, 'Test': df}
        """
        self.corpora = {}
        variant_keys = self._resolve_keys_for_variant(variant)

        for variant_key in variant_keys:
            ds_lbl = self.ds_label(variant_key)

            # ---- get & normalize splits ----
            train_df = self.normalise_data(self.dash_data[variant_key].train_data)
            test_df  = self.normalise_data(self.dash_data[variant_key].analysis_data)

            # ensure expected columns exist
            if "Core Tokens" not in train_df.columns and "core_tokens" in train_df.columns:
                train_df = train_df.rename(columns={"core_tokens": "Core Tokens"})
            if "Core Tokens" not in test_df.columns and "core_tokens" in test_df.columns:
                test_df  = test_df.rename(columns={"core_tokens": "Core Tokens"})

            self.corpora[ds_lbl] = {
                "splits": {
                    "Train": train_df,
                    "Test":  test_df,
                }
            }

        return self.corpora
    
    def iter_splits(self, ds_key: str, only_test: bool = False) -> Iterator[Tuple[str, str, pd.DataFrame]]:
        """
        Yield dataset splits.
        Set only_test=True to restrict to Test split (ignore Train).
        """
        splits = ("Test",) if only_test else ("Train", "Test")
        for sk in splits:
            if sk in self.corpora[ds_key]["splits"]:
                yield sk, sk, self.corpora[ds_key]["splits"][sk]




class DatasetStatsHelper(BaseDatasetHelper):
    @staticmethod
    def _iter_tokens_and_tags(sentences):
        for s in sentences:
            for w, t in zip(s["words"], s["tags"]):
                yield w, t

    def _compute_split_counts(self, split_sentences):
        # per-split token + NE counts (and per-split uniques)
        tokens = []
        ne_tokens = []
        ne_unique = set()
        for w, t in self._iter_tokens_and_tags(split_sentences):
            tokens.append(w)
            if t != "O":
                ne_tokens.append(w)
                ne_unique.add(w)
        total = len(tokens)
        unique_words = len(set(tokens))
        twr  = (unique_words / total) if total else 0.0          # ← TWR
        ne_words = len(ne_tokens)
        unique_ne_words = len(ne_unique)
        ne_prop = (ne_words / total) if total else 0.0
        ne_type_prop = (unique_ne_words / unique_words) if ne_words else 0.0
        tewr = (unique_ne_words / ne_words) if ne_words else 0.0                       # ← TEWR
        return {
            "Total Words": total,
            "NE Words": ne_words,
            "NE Proportion": ne_prop,
            "TWR": twr,                       # Type–Word Ratio (lexical diversity)
            "Unique Words": unique_words,
            "Unique NE Words": unique_ne_words,
            "NE Type Proportion": ne_type_prop,
            "TEWR": tewr,                     # NE Type–Word Ratio (NE diversity)
        }

    def generate_df(self, selected_variant: str, include_total: bool = True) -> pd.DataFrame:
        """
        Rows (index):
          - Total Words, Unique Words, NE Words, Unique NE Words, NE Proportion
        Cols:
          - MultiIndex (Language, Split[Train/Test/(Total)])
        """
        metrics = ["Total Words", "NE Words", "NE Proportion", "TWR", "Unique Words", "Unique NE Words", "NE Type Proportion", "TEWR"]
        cols, data = [], {m: [] for m in metrics}

        for ds_key in self.resolve_language_keys(selected_variant):
            ds_lbl = self.lang_label(ds_key)

            # We'll also accumulate across all splits to recompute 'Total' uniques properly
            all_tokens = []
            all_ne_tokens = []
            all_ne_unique = set()

            # per-split
            for sk, sk_lbl, split_sentences in self.iter_splits(ds_key):
                stats = self._compute_split_counts(split_sentences)
                cols.append((ds_lbl, sk_lbl))
                for m in metrics:
                    data[m].append(stats[m])

                # accumulate for overall recomputation
                for s in split_sentences:
                    all_tokens.extend(s["words"])
                    for w, t in zip(s["words"], s["tags"]):
                        if t != "O":
                            all_ne_tokens.append(w)
                            all_ne_unique.add(w)

            if include_total:
                total_tokens = len(all_tokens)
                unique_words = len(set(all_tokens))
                twr  = (unique_words / total_tokens) if total_tokens else 0.0          # ← TWR
                ne_words = len(all_ne_tokens)
                unique_ne_words = len(all_ne_unique)
                ne_prop = (ne_words / total_tokens) if total_tokens else 0.0
                ne_type_prop = (unique_ne_words / unique_words) if ne_words else 0.0
                tewr = (unique_ne_words / ne_words) if ne_words else 0.0                       # ← TEWR

                cols.append((ds_lbl, "Total"))
                totals_row = {
                    "Total Words": total_tokens,
                    "NE Words": ne_words,
                    "NE Proportion": ne_prop,
                    "TWR": twr,                       # Type–Word Ratio (lexical diversity)
                    "Unique Words": unique_words,
                    "Unique NE Words": unique_ne_words,
                    "NE Type Proportion": ne_type_prop,
                    "TEWR": tewr,                     # NE Type–Word Ratio (NE diversity)
                }
                for m in metrics:
                    data[m].append(totals_row[m])

        df = pd.DataFrame(data, index=pd.RangeIndex(len(cols))).T
        df.columns = pd.MultiIndex.from_tuples(cols, names=["Language", "Split"])
        return df

    

class EntityTagDistribution(BaseDatasetHelper):
    def generate_df(self, variant: str, tag_set: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Build a DataFrame with per-tag statistics.

        Columns:
            Split                : Split label (e.g. Train, Test).
            Tag                  : Entity tag (B-LOC, I-LOC, etc.).
            Tag Words            : Raw NE token count for this tag.
            Tag Types            : Unique word *forms* that appeared with this tag.
            TWR                  : Type–Word Ratio = Tag Types / Tag Words.
            Words Proportion     : Share of NE tokens in this tag
                                (denominator = all NE tokens in the split).
            Type Proportion      : Share of unique NE word *forms* in this tag
                                (denominator = union of all NE forms in the split).
            Tag Type Proportion  : Share of Tag Types in this tag
                                (denominator = sum of Tag Types across all tags in this split;
                                always sums to 1 per split).
            Dataset              : Dataset label (e.g. ANERCorp, CoNLL-2003).
        """
        use_tags = tag_set or _DEFAULT_TAGS
        rows = []
        for ds_key in self.resolve_language_keys(variant):
            ds_lbl = self.lang_label(ds_key)
            for _, sk_lbl, split_sentences in self.iter_splits(ds_key):
                use_tags = tag_set or None  # if None, _tag_stats_for_split derives locally
                # split_df = self._tag_stats_for_split(sk_lbl, split_sentences, use_tags)
                split_df = self._tag_stats_for_split(
                    split_name=sk_lbl,
                    split_sentences=split_sentences,
                    tag_set=use_tags
                )
                split_df["Language"] = ds_lbl
                # rows.append(split_df)
            
                # --- add Tag Type Proportion directly ---
                total_tag_types = split_df["Tag Types"].sum()
                if total_tag_types > 0:
                    split_df["Tag Type Proportion"] = split_df["Tag Types"] / total_tag_types
                else:
                    split_df["Tag Type Proportion"] = 0.0

                rows.append(split_df)
                

        return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(
            columns=[
                "Split", "Tag", 
                "Tag Words", "Tag Types", "TWR",                  
                "Words Proportion", "Type Proportion",      
                "Tag Type Proportion",   
                "Language"
                ]
        )
    
    def add_tag_type_proportion(df: pd.DataFrame) -> pd.DataFrame:
        totals = (
            df.groupby(["Language", "Split"], as_index=False)["Tag Types"]
            .sum()
            .rename(columns={"Tag Types": "Total Tag Types"})
        )
        out = df.merge(totals, on=["Language", "Split"], how="left")
        out["Tag Type Proportion"] = (out["Tag Types"] / out["Total Tag Types"]).fillna(0.0)
        return out

    def _tag_stats_for_split(
        self,
        split_name: str,
        split_sentences: List[Dict],
        *,
        tag_set: Optional[List[str]] = None,
        round_to: int = 2,
    ) -> pd.DataFrame:
        """
        If tag_set is None → discover tags just from this split.
        Otherwise → use the provided tag_set, filling in zeros for missing tags.
        """
        # derive tag_set if not passed
        if tag_set is None:
            tags = {self._normalize_tag(t) for sent in split_sentences for t in sent["tags"] if t != "O"}
            tag_set = sorted(tags)

        tag_counts = {t: 0 for t in tag_set}
        unique_tag_words = {t: set() for t in tag_set}

        total_ne_tokens = 0
        total_ne_unique_words = set()

        for sent in split_sentences:
            for w, t in zip(sent["words"], sent["tags"]):
                t = self._normalize_tag(t)
                if t != "O":
                    total_ne_tokens += 1
                    total_ne_unique_words.add(w)
                if t in tag_counts:
                    tag_counts[t] += 1
                    unique_tag_words[t].add(w)

        rows = []
        for t in tag_set:
            count = tag_counts[t]
            types = len(unique_tag_words[t])
            twr = (types / count) if count else 0.0
            token_prop = (count / total_ne_tokens) if total_ne_tokens else 0.0
            type_prop  = (types / len(total_ne_unique_words)) if total_ne_unique_words else 0.0
            rows.append({
                "Split": split_name,
                "Tag": t,
                "Tag Words": count,
                "Tag Types": types,
                "TWR": round(twr, round_to),
                "Words Proportion": round(token_prop, round_to),
                "Type Proportion": round(type_prop, round_to),
            })
        return pd.DataFrame(rows)


class EntitySpanDistribution(BaseDatasetHelper):
    """
    Builds span-level entity distributions.
    Returns columns:
      Dataset, Split, Scheme, Entity, Span Count, Span Proportion
    """

    def generate_df(
        self,
        variant: str,
        *,
        schemes: List[str] = ("IOB1", "IOB2"),
        splits: Optional[List[str]] = None,                # None → use iter_splits order
        entity_set: Optional[List[str]] = None,            # None → _DEFAULT_ENTITY_SET
        round_to: int = 2,
    ) -> pd.DataFrame:

        entity_span_set = entity_set or _DEFAULT_ENTITY_SPANS
        rows: List[Dict] = []

        for ds_key in self.resolve_language_keys(variant):
            ds_lbl = self.lang_label(ds_key)

            split_iter = list(self.iter_splits(ds_key))
            if splits:
                split_iter = [(sk, sk_lbl, sents) for (sk, sk_lbl, sents) in split_iter if sk in splits]

            for sk, sk_lbl, split_sentences in split_iter:
                # gold tags for this split
                y_true = [sen["tags"] for sen in split_sentences]

                for version in schemes:
                    # --- keep YOUR original extraction paths exactly ---
                    if version == "IOB1":
                        true_entities = get_entities(y_true)
                        # true_entities: list of tuples; count by entity[0]
                    else:
                        scheme = auto_detect(y_true, False)
                        entities = Entities(y_true, scheme, False)
                        # flatten per-sentence entities; to_tuple() → (sent_id, type, start, end)
                        true_entities = [entity.to_tuple()[1:] for sen in entities.entities for entity in sen]

                    # normalise labels and count only what we care about
                    raw_counts = Counter(
                        _ENTITY_NORMALIZE.get(ent[0], ent[0]) for ent in true_entities
                    )

                    # filter to requested entity_set (keeps zero rows consistent)
                    counts_in_set = {e: raw_counts.get(e, 0) for e in entity_span_set}
                    total = sum(counts_in_set.values())

                    for entity_span in entity_span_set:
                        count = counts_in_set[entity_span]
                        prop = (count / total) if total else 0.0
                        rows.append({
                            "Language": ds_lbl,            # e.g., ANERCorp / CoNLL-2003
                            "Split": sk_lbl,                  # keep lower-case like your other tables
                            "Scheme": version,            # IOB1 / IOB2
                            "Entity": entity_span,        # PER / LOC / ORG / MISC
                            "Span Count": count,
                            "Span Proportion": round(prop, round_to),
                        })

        return pd.DataFrame(rows, columns=[
            "Language", "Split", "Scheme", "Entity", "Span Count", "Span Proportion"
        ])


class EntitySpanComplexity(BaseDatasetHelper):
    """
    Produce one row per entity span:
      Columns: Language, Split, Scheme, Entity, Span Length
    Use this directly with a faceted BOX PLOT.
    """

    def generate_df(
        self,
        variant: str,
        *,
        schemes=("IOB1","IOB2"),
        splits=None,                    # e.g. ["train","test"] or None → iter_splits order
        strict_iob2: bool = True,       # how you compute length for IOB2: end-start(+1)
    ) -> pd.DataFrame:
        rows = []

        for ds_key in self.resolve_language_keys(variant):
            ds_lbl = self.lang_label(ds_key)

            split_iter = list(self.iter_splits(ds_key))
            if splits:
                split_iter = [(sk, sk_lbl, sents) for (sk, sk_lbl, sents) in split_iter if sk in splits]

            for sk, sk_lbl, split_sentences in split_iter:
                y_true = [sen["tags"] for sen in split_sentences]

                for scheme_name in schemes:
                    if scheme_name == "IOB1":
                        # Your original path
                        # get_entities returns list of (type, start, end) across corpus
                        true_entities = get_entities(y_true)
                        # inclusive length for IOB1 (matches your previous strict=False):
                        for ent_type, start, end in true_entities:
                            ent_type = _ENTITY_NORMALIZE.get(ent_type, ent_type)
                            length = (end - start + 1)
                            rows.append({
                                "Language": ds_lbl,
                                "Split": sk_lbl,                 # keep lower-case for consistency
                                "Scheme": scheme_name,
                                "Entity": ent_type,
                                "Span Length": int(length),
                            })
                    else:
                        # Your original path for IOB2
                        scheme = auto_detect(y_true, False)
                        ent_obj = Entities(y_true, scheme, False)
                        # per sentence entities; to_tuple() = (sent_id, type, start, end)
                        for sen in ent_obj.entities:
                            for e in sen:
                                _, ent_type, start, end = e.to_tuple()
                                ent_type = _ENTITY_NORMALIZE.get(ent_type, ent_type)
                                if strict_iob2:
                                    # strict=True in your earlier class meant end-start (exclusive end)
                                    # but your text says: “mean span lengths … average number of tokens”
                                    # For tokens, inclusive is standard, so mirror IOB1:
                                    length = (end - start + 1)
                                else:
                                    length = (end - start)
                                rows.append({
                                    "Language": ds_lbl,
                                    "Split": sk_lbl,
                                    "Scheme": scheme_name,
                                    "Entity": ent_type,
                                    "Span Length": int(length),
                                })

        return pd.DataFrame(rows, columns=["Language", "Split", "Scheme", "Entity", "Span Length"])


class WordTypeFrequencyDistribution(BaseDatasetHelper):
    """
    For each (Language, Split, Tag), compute word-type frequency stats:
      - Mean Frequency
      - Median Frequency
      - Standard Deviation
    Word-type frequencies are counts of unique word forms appearing with that tag
    within the split (e.g., how many times each distinct name tagged B-PER appears).
    """

    def __init__(self, corpora: Dict, tags_of_interest: Optional[List[str]] = None):
        super().__init__(corpora)
        self.tags = tags_of_interest or _DEFAULT_TAGS

    def generate_df(self, variant: str, round_to: int = 2, splits: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Returns a tidy DataFrame with columns:
          Language, Split, Tag, Mean Frequency, Median Frequency, Standard Deviation
        """
        rows: List[dict] = []

        for ds_key in self.resolve_language_keys(variant):
            ds_lbl = self.lang_label(ds_key)

            # walk the available splits with your built-in iterator
            for split_key, split_lbl, split_sentences in self.iter_splits(ds_key):
                # if the caller restricts splits, skip others
                if splits and split_key not in splits:
                    continue

                # count word frequencies per tag for THIS split
                per_tag_counters = {tag: Counter() for tag in self.tags}
                for sent in split_sentences:
                    for w, t in zip(sent["words"], sent["tags"]):
                        t = self._normalize_tag(t)  # PERS→PER etc.
                        if t in per_tag_counters:
                            per_tag_counters[t][w] += 1

                # turn per-tag counters into stats
                for tag, counter in per_tag_counters.items():
                    freqs = np.fromiter(counter.values(), dtype=np.int64) if counter else np.array([], dtype=np.int64)
                    if freqs.size:
                        mean_f = float(np.mean(freqs))
                        med_f  = float(np.median(freqs))
                        std_f  = float(np.std(freqs, ddof=0))
                    else:
                        mean_f = med_f = std_f = 0.0

                    rows.append({
                        "Language": ds_lbl,
                        "Split": split_lbl,
                        "Tag": tag,
                        "Mean Frequency": round(mean_f, round_to),
                        "Median Frequency": round(med_f, round_to),
                        "Standard Deviation": round(std_f, round_to),
                    })

        return pd.DataFrame(rows)
    



class DatasetOOVRate(BaseDatasetHelper):
    """Rows = datasets, Cols = metrics (matches your example)."""

    def generate_df(self, selected_variant: str, round_to: int = 4) -> pd.DataFrame:
        rows = []
        seen_labels = set()

        for ds_key in self.resolve_language_keys(selected_variant):
            ds_lbl = self.lang_label(ds_key)
            if ds_lbl in seen_labels:
                continue
            seen_labels.add(ds_lbl)

            splits = self.corpora[ds_key]["splits"]
            train = splits.get("train")
            test  = splits.get("test")

            if not train or not test:
                rows.append({
                    "Language": ds_lbl,
                    "OOV Words Count": 0.0,
                    "Total Unique Words in Test": 0.0,
                    "OOV Rate": 0.0,
                })
                continue

            train_words = {w for s in train for w in s["words"]}
            test_words  = {w for s in test  for w in s["words"]}
            oov_words   = test_words - train_words
            oov_rate    = (len(oov_words) / len(test_words)) if test_words else 0.0

            rows.append({
                "Language": ds_lbl,
                "OOV Words Count": float(len(oov_words)),
                "Total Unique Words in Test": float(len(test_words)),
                "OOV Rate": round(oov_rate, round_to),
            })

        return pd.DataFrame(
            rows,
            columns=["Language", "OOV Words Count", "Total Unique Words in Test", "OOV Rate"]
        )


class EntityTagOOVRate(BaseDatasetHelper):
    """
    Per-tag OOV, computed train→test for each dataset.
    Output rows: Dataset, Tag, OOV Words Count, Total Unique Words in Test, OOV Rate
    """

    def __init__(self, corpora: Dict, tags: Optional[List[str]] = None):
        super().__init__(corpora)
        self.tags = tags or _DEFAULT_TAGS

    def _unique_words_per_tag(self, split_sentences, tags: List[str]) -> Dict[str, set]:
        buckets = {t: set() for t in tags}
        for s in split_sentences:
            for w, t in zip(s["words"], s["tags"]):
                t = self._normalize_tag(t)  # e.g., PERS→PER
                if t in buckets:
                    buckets[t].add(w)
        return buckets

    def generate_df(self, selected_variant: str, round_to: int = 3) -> pd.DataFrame:
        rows: List[dict] = []
        seen_labels = set()

        for ds_key in self.resolve_language_keys(selected_variant):
            ds_lbl = self.lang_label(ds_key)
            if ds_lbl in seen_labels:
                continue
            seen_labels.add(ds_lbl)

            splits = self.corpora[ds_key]["splits"]
            train = splits.get("train")
            test  = splits.get("test")
            if not train or not test:
                # skip silently; or emit zeros if you prefer
                continue

            train_buckets = self._unique_words_per_tag(train, self.tags)
            test_buckets  = self._unique_words_per_tag(test,  self.tags)

            for tag in self.tags:
                test_set  = test_buckets.get(tag, set())
                train_set = train_buckets.get(tag, set())
                oov_set   = test_set - train_set
                oov_rate  = (len(oov_set) / len(test_set)) if test_set else 0.0
                rows.append({
                    "Language": ds_lbl,
                    "Tag": tag,
                    "OOV Words Count": float(len(oov_set)),
                    "Total Unique Words in Test": float(len(test_set)),
                    "OOV Rate": round(oov_rate, round_to),
                })

        return pd.DataFrame(rows, columns=[
            "Language", "Tag", "OOV Words Count", "Total Unique Words in Test", "OOV Rate"
        ])



class TokenisedDatasetStatsHelper(BaseDashDataProcessor):
    """
    Tokens-only stats from DF-based corpora built by BaseDashDataProcessor.build_corpora().
    Expects columns: 'Core Tokens', 'True Labels'.
    Output: rows = metrics, cols = (Language, Split[Train/Test/(Total)]).
    """

    @staticmethod
    def _compute_from_df(df: pd.DataFrame) -> dict:
        if "Core Tokens" not in df.columns or "True Labels" not in df.columns:
            raise KeyError("Expected 'Core Tokens' and 'True Labels' in split DataFrame.")

        total_tokens = int(len(df))
        unique_tokens = int(df["Core Tokens"].nunique())

        is_ne = df["True Labels"].astype(str) != "O"
        ne_tokens = int(is_ne.sum())
        unique_ne_tokens = int(df.loc[is_ne, "Core Tokens"].nunique())

        twr  = (unique_tokens / total_tokens) if total_tokens else 0.0
        ne_prop = (ne_tokens / total_tokens) if total_tokens else 0.0
        tewr = (unique_ne_tokens / ne_tokens) if ne_tokens else 0.0

        return {
            "Total Tokens": total_tokens,
            "Unique Tokens": unique_tokens,
            "NE Tokens": ne_tokens,
            "Unique NE Tokens": unique_ne_tokens,
            "Tokens NE Proportion": ne_prop,
            "TTR": twr,
            "TETR": tewr,
        }

    def generate_df(self, selected_variant: str, include_total: bool = True) -> pd.DataFrame:
        # 1) build corpora for this variant
        self.build_corpora(selected_variant)

        metrics = [
            "Total Tokens", "NE Tokens", "Tokens NE Proportion",
            "TTR", "Unique Tokens", "Unique NE Tokens", "TETR",
        ]
        cols, data = [], {m: [] for m in metrics}

        # 2) iterate the corpora dict (Language → splits)
        for dataset_name, content in self.corpora.items():
            split_dict = content["splits"]
            parts = []

            for split_name, split_df in split_dict.items():  # 'Train' / 'Test'
                stats = self._compute_from_df(split_df)
                cols.append((dataset_name, split_name))
                for m in metrics:
                    data[m].append(stats[m])
                parts.append(split_df[["Core Tokens", "True Labels"]])

            if include_total and parts:
                merged = pd.concat(parts, ignore_index=True)
                totals = self._compute_from_df(merged)
                cols.append((dataset_name, "Total"))
                for m in metrics:
                    data[m].append(totals[m])

        # 3) table shape like your other helpers
        df = pd.DataFrame(data, index=pd.RangeIndex(len(cols))).T
        df.columns = pd.MultiIndex.from_tuples(cols, names=["Language", "Split"])
        return df


class EntityTagTokenTypeDistribution(BaseDashDataProcessor):
    def generate_df(self, selected_variant: str) -> pd.DataFrame:
        self.build_corpora(selected_variant)
        rows = []

        for ds_lbl, content in self.corpora.items():
            for split_key, _, df in self.iter_splits(ds_lbl):
                df_ne = df[df["True Labels"] != "O"].copy()
                if df_ne.empty:
                    continue
                if "Token Ids" not in df_ne.columns:
                    raise KeyError("Expected 'Token Ids' column for token-type analysis.")

                # no tag filter here

                total_ne_tokens = int(len(df_ne))
                all_ne_unique_token_types = int(df_ne["Token Ids"].nunique())

                per_tag = (
                    df_ne.groupby("True Labels", dropna=False)
                        .agg(**{
                            "Total Tokens": ("Token Ids", "count"),
                            "Tag Types": ("Token Ids", pd.Series.nunique),
                        })
                        .reset_index()
                        .rename(columns={"True Labels": "Tag"})
                )

                per_tag["TTR"] = np.where(per_tag["Total Tokens"] > 0,
                                          per_tag["Tag Types"] / per_tag["Total Tokens"], 0.0)
                per_tag["Tokens Proportion"] = np.where(total_ne_tokens > 0,
                                                        per_tag["Total Tokens"] / total_ne_tokens, 0.0)
                per_tag["Type Proportion"] = np.where(all_ne_unique_token_types > 0,
                                                      per_tag["Tag Types"] / all_ne_unique_token_types, 0.0)
                per_tag["Split"] = split_key
                per_tag["Language"] = ds_lbl
                rows.append(per_tag)

        return (pd.concat(rows, ignore_index=True) if rows else
                pd.DataFrame(columns=["Split","Tag","Total Tokens","Tag Types","TTR","Tokens Proportion","Type Proportion","Language"]))

class TokenTypeFrequencyDistribution(BaseDashDataProcessor):
    def generate_df(self, selected_variant: str, round_to: int = 2) -> pd.DataFrame:
        self.build_corpora(selected_variant)
        rows = []

        for ds_lbl, content in self.corpora.items():
            for split_key, _, df in self.iter_splits(ds_lbl):
                df_ne = df[df["True Labels"] != "O"].copy()
                if df_ne.empty:
                    continue
                if "Token Ids" not in df_ne.columns:
                    raise KeyError("Expected 'Token Ids' column for token-type analysis.")

                # no tag filter here

                per_tag_counters = {
                    tag: Counter(g["Token Ids"])
                    for tag, g in df_ne.groupby("True Labels")
                }

                for tag, counter in per_tag_counters.items():
                    freqs = np.fromiter(counter.values(), dtype=np.int64) if counter else np.array([], dtype=np.int64)
                    if freqs.size:
                        mean_f = float(np.mean(freqs))
                        med_f  = float(np.median(freqs))
                        std_f  = float(np.std(freqs, ddof=0))
                    else:
                        mean_f = med_f = std_f = 0.0

                    rows.append({
                        "Language": ds_lbl,
                        "Split": split_key,
                        "Tag": tag,
                        "Mean Frequency": round(mean_f, round_to),
                        "Median Frequency": round(med_f, round_to),
                        "Standard Deviation": round(std_f, round_to),
                    })
        return pd.DataFrame(rows)




class DatasetTokenOOVRate(BaseDashDataProcessor):
    """
    One row per language:
      - OOV Core Tokens Count
      - Total Unique Core Tokens in Test
      - OOV Rate = count / total (decimal, not %)
    """

    def generate_df(self, selected_variant: str, round_to: int = 4) -> pd.DataFrame:
        self.build_corpora(selected_variant)

        rows = []
        for ds_lbl, content in self.corpora.items():
            train = content["splits"]["Train"]
            test  = content["splits"]["Test"]

            # unique sets (all tokens, not per tag)
            train_set = set(train["Core Tokens"].dropna().astype(str).unique())
            test_set  = set(test["Core Tokens"].dropna().astype(str).unique())

            oov = test_set - train_set
            denom = len(test_set)
            rate = (len(oov) / denom) if denom else 0.0

            rows.append({
                "Language": ds_lbl,
                "OOV Core Tokens Count": len(oov),
                "Total Unique Core Tokens in Test": denom,
                "OOV Rate": round(rate, round_to),
            })

        # wide → to match your simple table shape (no splits)
        if not rows:
            return pd.DataFrame(columns=["OOV Core Tokens Count","Total Unique Core Tokens in Test","OOV Rate"])

        df_long = pd.DataFrame(rows)
        df_long.set_index("Dataset", inplace=True)
        # return tidy wide (datasets as columns)
        return df_long.T  # rows = metrics; cols = datasets


class EntityTagTokenOOVRate(BaseDashDataProcessor):
    """
    One row per language × tag:
      - OOV Core Tokens Count
      - Total Unique Core Tokens in Test
      - OOV Rate (decimal)
    """

    @staticmethod
    def _per_tag_sets(df: pd.DataFrame) -> Dict[str, set]:
        """Return {tag -> set(unique Core Tokens)} for NE tags (True Labels != 'O')."""
        if "True Labels" not in df.columns or "Core Tokens" not in df.columns:
            raise KeyError("Expected columns 'True Labels' and 'Core Tokens'.")
        df_ne = df[df["True Labels"] != "O"][["True Labels", "Core Tokens"]].dropna()
        buckets: Dict[str, set] = {}
        for tag, g in df_ne.groupby("True Labels"):
            buckets[tag] = set(g["Core Tokens"].astype(str).unique())
        return buckets

    def generate_df(self, selected_variant: str, round_to: int = 3) -> pd.DataFrame:
        self.build_corpora(selected_variant)
        rows = []

        for ds_lbl, content in self.corpora.items():
            train = content["splits"]["Train"]
            test  = content["splits"]["Test"]

            train_sets = self._per_tag_sets(train)
            test_sets  = self._per_tag_sets(test)

            # Use tags observed in TEST (typical OOV target); include train-only tags if you want union
            tags = sorted(test_sets.keys())

            for tag in tags:
                test_set  = test_sets.get(tag, set())
                train_set = train_sets.get(tag, set())
                oov_set   = test_set - train_set
                denom     = len(test_set)
                rate      = (len(oov_set) / denom) if denom else 0.0

                rows.append({
                    "Language": ds_lbl,
                    "Tag": tag,
                    "OOV Words Count": float(len(oov_set)),                 # keep names for compatibility
                    "Total Unique Words in Test": float(denom),
                    "OOV Rate": round(rate, round_to),                      # decimal (not %)
                })

        return pd.DataFrame(
            rows,
            columns=["Language", "Tag", "OOV Words Count", "Total Unique Words in Test", "OOV Rate"]
        )
    

class WordTypeOverlap(BaseDatasetHelper):
    """
    Compute overlap of unique word *types* across entity tags for sentence-based corpora.

    Options mirror the DF-based class:
      - include_O, tag_set, fill_diagonal, tag_order
    Tag normalisation uses BaseDatasetHelper._normalize_tag (PERS→PER etc.).
    """

    def _tag_sets_for_split(
        self,
        split_sentences: list[dict],
        *,
        include_O: bool,
        allowed_tags: Optional[set]
    ) -> dict[str, set]:
        buckets: dict[str, set] = {}
        for s in split_sentences:
            for w, t in zip(s["words"], s["tags"]):
                t = self._normalize_tag(str(t))
                if not include_O and t == "O":
                    continue
                if allowed_tags is not None and t not in allowed_tags:
                    continue
                buckets.setdefault(t, set()).add(str(w))
        return buckets

    @staticmethod
    def _overlap_matrix(buckets: dict[str, set], *, order: list[str], fill_diagonal: bool) -> pd.DataFrame:
        mat = pd.DataFrame(0, index=order, columns=order, dtype=int)
        for t1 in order:
            for t2 in order:
                if t1 == t2 and fill_diagonal:
                    mat.loc[t1, t2] = 0
                else:
                    mat.loc[t1, t2] = len(buckets.get(t1, set()) & buckets.get(t2, set()))
        return mat

    def _discover_tags_for_dataset(self, ds_key: str, include_O: bool) -> list[str]:
        tags = set()
        for _, _, split_sentences in self.iter_splits(ds_key):
            for s in split_sentences:
                for t in s["tags"]:
                    t = self._normalize_tag(str(t))
                    if not include_O and t == "O":
                        continue
                    tags.add(t)
        return sorted(tags)

    def generate_matrices(
        self,
        selected_variant: str,
        *,
        include_O: bool,
        tag_set: Optional[list[str]] = None,
        fill_diagonal: bool = True,
        tag_order: Optional[list[str]] = None,
    ) -> dict[str, dict[str, pd.DataFrame]]:
        """
        Returns:
          {DatasetLabel -> {SplitLabel -> pd.DataFrame(matrix)}}
        Note: This class expects `self.corpora` in the sentence-based shape you
              already use with BaseDatasetHelper (i.e., not built via BaseDashDataProcessor).
        """
        out: dict[str, dict[str, pd.DataFrame]] = {}

        for ds_key in self.resolve_language_keys(selected_variant):
            ds_lbl = self.lang_label(ds_key)

            # choose tag inventory
            if tag_set is None:
                tags = self._discover_tags_for_dataset(ds_key, include_O=include_O)
            else:
                tags = list(tag_set)
                if not include_O and "O" in tags:
                    tags = [t for t in tags if t != "O"]

            order = tag_order or tags
            out[ds_lbl] = {}

            for _, split_lbl, split_sentences in self.iter_splits(ds_key):
                buckets = self._tag_sets_for_split(
                    split_sentences,
                    include_O=include_O,
                    allowed_tags=set(order),
                )
                mat = self._overlap_matrix(buckets, order=order, fill_diagonal=fill_diagonal)
                out[ds_lbl][split_lbl] = mat

        return out

    def generate_df(
        self,
        selected_variant: str,
        *,
        include_O: bool = True,
        tag_set: Optional[list[str]] = None,
        fill_diagonal: bool = True,
        tag_order: Optional[list[str]] = None,
    ) -> pd.DataFrame:
        """
        Returns tidy long form:
          columns = [Language, Split, Tag1, Tag2, Overlap Count]
        """
        mats = self.generate_matrices(
            selected_variant,
            include_O=include_O,
            tag_set=tag_set,
            fill_diagonal=fill_diagonal,
            tag_order=tag_order,
        )
        rows = []
        for ds_lbl, splits in mats.items():
            for split_lbl, mat in splits.items():
                for t1 in mat.index:
                    for t2 in mat.columns:
                        rows.append({
                            "Language": ds_lbl,
                            "Split": split_lbl,
                            "Tag1": t1,
                            "Tag2": t2,
                            "Overlap Count": int(mat.loc[t1, t2]),
                        })
        return pd.DataFrame(rows, columns=["Language","Split","Tag1","Tag2","Overlap Count"])


class TokenTypeOverlap(BaseDashDataProcessor):
    """
    Compute overlap of unique token *types* across entity tags for DF-based corpora
    (produced by BaseDashDataProcessor.build_corpora()).

    Options:
      - include_O: include the 'O' tag as one bucket
      - tag_set:   fixed tag inventory; if None, derive from data per dataset (union of splits)
      - fill_diagonal: set diagonal to 0 (typical for overlap visuals)
      - tag_order: explicit ordering; else sorted discovered tags
    """

    def _tag_sets_for_split(
        self, df: pd.DataFrame, *, include_O: bool, allowed_tags: Optional[set]
    ) -> dict[str, set]:
        if "Core Tokens" not in df.columns or "True Labels" not in df.columns:
            raise KeyError("Expected columns 'Core Tokens' and 'True Labels'.")

        df = df[["Core Tokens", "True Labels"]].dropna().copy()
        df["True Labels"] = df["True Labels"].astype(str)

        if not include_O:
            df = df[df["True Labels"] != "O"]

        if allowed_tags is not None:
            df = df[df["True Labels"].isin(allowed_tags)]

        buckets: dict[str, set] = {}
        for tag, g in df.groupby("True Labels", dropna=False):
            buckets[tag] = set(g["Core Tokens"].astype(str).unique())
        return buckets

    @staticmethod
    def _overlap_matrix(buckets: dict[str, set], *, order: list[str], fill_diagonal: bool) -> pd.DataFrame:
        mat = pd.DataFrame(0, index=order, columns=order, dtype=int)
        for t1 in order:
            for t2 in order:
                if t1 == t2 and fill_diagonal:
                    mat.loc[t1, t2] = 0
                else:
                    s1 = buckets.get(t1, set())
                    s2 = buckets.get(t2, set())
                    mat.loc[t1, t2] = len(s1 & s2)
        return mat

    def _discover_tags_for_dataset(self, ds_lbl: str, include_O: bool) -> list[str]:
        tags = set()
        for split_key, _, df in self.iter_splits(ds_lbl):
            vals = set(df["True Labels"].dropna().astype(str).unique())
            if not include_O:
                vals.discard("O")
            tags |= vals
        return sorted(tags)

    def generate_matrices(
        self,
        selected_variant: str,
        *,
        include_O: bool,
        tag_set: Optional[list[str]] = None,
        fill_diagonal: bool = True,
        tag_order: Optional[list[str]] = None,
    ) -> dict[str, dict[str, pd.DataFrame]]:
        """
        Returns:
          {Language -> {Split -> pd.DataFrame(matrix)}}
        """
        self.build_corpora(selected_variant)

        out: dict[str, dict[str, pd.DataFrame]] = {}
        for ds_lbl, content in self.corpora.items():
            # choose tag inventory
            if tag_set is None:
                tags = self._discover_tags_for_dataset(ds_lbl, include_O=include_O)
            else:
                tags = list(tag_set)
                if not include_O and "O" in tags:
                    tags = [t for t in tags if t != "O"]

            order = tag_order or tags
            out[ds_lbl] = {}

            for split_key, _, df in self.iter_splits(ds_lbl):
                buckets = self._tag_sets_for_split(df, include_O=include_O, allowed_tags=set(order))
                mat = self._overlap_matrix(buckets, order=order, fill_diagonal=fill_diagonal)
                out[ds_lbl][split_key] = mat

        return out

    def generate_df(
        self,
        selected_variant: str,
        *,
        include_O: bool = True,
        tag_set: Optional[list[str]] = None,
        fill_diagonal: bool = True,
        tag_order: Optional[list[str]] = None,
    ) -> pd.DataFrame:
        """
        Returns tidy long form:
          columns = [Language, Split, Tag1, Tag2, Overlap Count]
        """
        mats = self.generate_matrices(
            selected_variant,
            include_O=include_O,
            tag_set=tag_set,
            fill_diagonal=fill_diagonal,
            tag_order=tag_order,
        )
        rows = []
        for ds_lbl, splits in mats.items():
            for split_lbl, mat in splits.items():
                for t1 in mat.index:
                    for t2 in mat.columns:
                        rows.append({
                            "Language": ds_lbl,
                            "Split": split_lbl,
                            "Tag1": t1,
                            "Tag2": t2,
                            "Overlap Count": int(mat.loc[t1, t2]),
                        })
        return pd.DataFrame(rows, columns=["Language","Split","Tag1","Tag2","Overlap Count"])




class TokenizationRateHelper(BaseDashDataProcessor):
    """
    Generates tokenisation rate per entity tag.
    Outputs Language × Split × Tag with Mean, Std, and Text Label.
    """

    COL = "Tokenization Rate"

    @staticmethod
    def _summarise(split_df: pd.DataFrame, value_col: str, round_to: int) -> pd.DataFrame:
        if value_col not in split_df.columns or "True Labels" not in split_df.columns:
            return pd.DataFrame(columns=["Tag", "Mean Value", "Std Dev", "Text Label"])

        df = split_df[["True Labels", value_col]].copy()
        df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
        df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[value_col])

        if df.empty:
            return pd.DataFrame(columns=["Tag", "Mean Value", "Std Dev", "Text Label"])

        g = (
            df.groupby("True Labels", as_index=False)[value_col]
              .agg(**{"Mean Value": "mean", "Std Dev": "std"})
              .rename(columns={"True Labels": "Tag"})
        )
        g["Std Dev"] = g["Std Dev"].fillna(0.0)
        g["Mean Value"] = g["Mean Value"].round(round_to)
        g["Std Dev"]    = g["Std Dev"].round(round_to)

        g["Text Label"] = g.apply(
            lambda r: f"{r['Mean Value']:.{round_to}f} <br>±<br> {r['Std Dev']:.{round_to}f}",
            axis=1
        )
        return g[["Tag", "Mean Value", "Std Dev", "Text Label"]]

    def generate_df(self, selected_variant: str, round_to: int = 3) -> pd.DataFrame:
        self.build_corpora(selected_variant)

        rows = []
        for ds_lbl, content in self.corpora.items():
            for split_key, _, split_df in self.iter_splits(ds_lbl, only_test=True):
                out = self._summarise(split_df, self.COL, round_to)
                if not out.empty:
                    out["Language"] = ds_lbl
                    out["Split"]   = split_key
                    rows.append(out)

        if not rows:
            return pd.DataFrame(columns=["Language", "Split", "Tag", "Mean Value", "Std Dev", "Text Label"])

        return pd.concat(rows, ignore_index=True)[
            ["Language", "Split", "Tag", "Mean Value", "Std Dev", "Text Label"]
        ]

    


class AmbiguityHelper(BaseDashDataProcessor):
    """
    Merge Token and Word ambiguity at entity level, per Language × Split.
    Output columns:
      Language, Split, Tag, Level ('Token Level'|'Word Level'),
      Mean Value, Std Dev, Text Label
    """
    TOKEN_COL = "Token Ambiguity"
    WORD_COL  = "Word Ambiguity"

    @staticmethod
    def _summarise(split_df: pd.DataFrame, value_col: str, level_name: str, round_to: int) -> pd.DataFrame:
        if value_col not in split_df.columns:
            return pd.DataFrame(columns=["Tag", "Level", "Mean Value", "Std Dev", "Text Label"])

        df = split_df.loc[split_df[value_col] != -1, ["True Labels", value_col]].copy()
        if df.empty:
            return pd.DataFrame(columns=["Tag", "Level", "Mean Value", "Std Dev", "Text Label"])

        g = (
            df.groupby("True Labels", as_index=False)[value_col]
              .agg(["mean", "std"])
              .reset_index()
              .rename(columns={"True Labels": "Tag", "mean": "Mean Value", "std": "Std Dev"})
        )
        g["Mean Value"] = pd.to_numeric(g["Mean Value"], errors="coerce").round(round_to)
        g["Std Dev"]    = pd.to_numeric(g["Std Dev"],    errors="coerce").round(round_to)
        g["Level"]      = level_name
        g["Text Label"] = g.apply(
            lambda r: f"{(r['Mean Value'] if pd.notna(r['Mean Value']) else 0):.{round_to}f}"
                      f" <br>±<br>"
                      f"{(r['Std Dev'] if pd.notna(r['Std Dev']) else 0):.{round_to}f}",
            axis=1
        )
        return g[["Tag", "Level", "Mean Value", "Std Dev", "Text Label"]]

    def generate_df(self, selected_variant: str, round_to: int = 3) -> pd.DataFrame:
        self.build_corpora(selected_variant)

        rows = []
        for ds_lbl, content in self.corpora.items():
            for split_key, _, split_df in self.iter_splits(ds_lbl, only_test=True):
                # Token-level ambiguity
                tok = self._summarise(split_df, self.TOKEN_COL, "Token Level", round_to)
                if not tok.empty:
                    tok["Language"] = ds_lbl
                    tok["Split"]   = split_key
                    rows.append(tok)

                # Word-level ambiguity
                wrd = self._summarise(split_df, self.WORD_COL, "Word Level", round_to)
                if not wrd.empty:
                    wrd["Language"] = ds_lbl
                    wrd["Split"]   = split_key
                    rows.append(wrd)

        if not rows:
            return pd.DataFrame(columns=[
                "Language", "Split", "Tag", "Level", "Mean Value", "Std Dev", "Text Label"
            ])

        out = pd.concat(rows, ignore_index=True)
        return out[["Language", "Split", "Tag", "Level", "Mean Value", "Std Dev", "Text Label"]]

class ConsistencyHelper(BaseDashDataProcessor):
    """
    Merge Consistency and Inconsistency ratios at entity level, per Language × Split.
    Output columns:
      Language, Split, Tag, Level ('Consistency Ratio'|'Inconsistency Ratio'),
      Mean Value, Std Dev, Text Label
    """
    CONSISTENCY_COL    = "Consistency Ratio"
    INCONSISTENCY_COL  = "Inconsistency Ratio"

    @staticmethod
    def _summarise(split_df: pd.DataFrame, value_col: str, level_name: str, round_to: int) -> pd.DataFrame:
        # guard: column present?
        if value_col not in split_df.columns or "True Labels" not in split_df.columns:
            return pd.DataFrame(columns=["Tag", "Level", "Mean Value", "Std Dev", "Text Label"])

        # keep valid numeric values only
        df = split_df[["True Labels", value_col]].copy()
        df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
        df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[value_col])

        if df.empty:
            return pd.DataFrame(columns=["Tag", "Level", "Mean Value", "Std Dev", "Text Label"])

        # aggregate by tag
        g = (
            df.groupby("True Labels", as_index=False)[value_col]
              .agg(**{"Mean Value": "mean", "Std Dev": "std"})  # pandas std -> ddof=1
              .rename(columns={"True Labels": "Tag"})
        )

        # handle single-sample std (NaN) cleanly
        g["Std Dev"] = g["Std Dev"].fillna(0.0)

        # rounding
        g["Mean Value"] = pd.to_numeric(g["Mean Value"], errors="coerce").round(round_to)
        g["Std Dev"]    = pd.to_numeric(g["Std Dev"],    errors="coerce").round(round_to)

        # annotate display fields
        g["Level"]      = level_name
        g["Text Label"] = g.apply(
            lambda r: f"{(r['Mean Value'] if pd.notna(r['Mean Value']) else 0):.{round_to}f}"
                      f" <br>±<br>"
                      f"{(r['Std Dev'] if pd.notna(r['Std Dev']) else 0):.{round_to}f}",
            axis=1
        )
        return g[["Tag", "Level", "Mean Value", "Std Dev", "Text Label"]]

    def generate_df(self, selected_variant: str, round_to: int = 3) -> pd.DataFrame:
        self.build_corpora(selected_variant)

        rows = []
        for ds_lbl, content in self.corpora.items():
            for split_key, _, split_df in self.iter_splits(ds_lbl, only_test=True):
                # Consistency
                con = self._summarise(split_df, self.CONSISTENCY_COL, "Consistency Ratio", round_to)
                if not con.empty:
                    con["Language"] = ds_lbl
                    con["Split"]   = split_key
                    rows.append(con)

                # Inconsistency
                inc = self._summarise(split_df, self.INCONSISTENCY_COL, "Inconsistency Ratio", round_to)
                if not inc.empty:
                    inc["Language"] = ds_lbl
                    inc["Split"]   = split_key
                    rows.append(inc)

        if not rows:
            return pd.DataFrame(columns=[
                "Language", "Split", "Tag", "Level", "Mean Value", "Std Dev", "Text Label"
            ])

        out = pd.concat(rows, ignore_index=True)
        return out[["Language", "Split", "Tag", "Level", "Mean Value", "Std Dev", "Text Label"]]


class LossHelper(BaseDashDataProcessor):
    COL = "Loss Values"

    @staticmethod
    def _summarise(split_df: pd.DataFrame, value_col: str, round_to: int) -> pd.DataFrame:
        if value_col not in split_df.columns or "True Labels" not in split_df.columns:
            return pd.DataFrame(columns=["Tag", "Mean Value", "Std Dev", "Text Label"])

        df = split_df[["True Labels", value_col]].copy()
        df[value_col] = pd.to_numeric(df[value_col], errors="coerce")
        df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[value_col])
        if df.empty:
            return pd.DataFrame(columns=["Tag", "Mean Value", "Std Dev", "Text Label"])

        g = (df.groupby("True Labels", as_index=False)[value_col]
               .agg(**{"Mean Value": "mean", "Std Dev": "std"})
               .rename(columns={"True Labels": "Tag"}))
        g["Std Dev"] = g["Std Dev"].fillna(0.0)
        g["Mean Value"] = g["Mean Value"].round(round_to)
        g["Std Dev"]    = g["Std Dev"].round(round_to)
        g["Text Label"] = g.apply(
            lambda r: f"{r['Mean Value']:.{round_to}f} <br>±<br> {r['Std Dev']:.{round_to}f}", axis=1
        )
        return g[["Tag", "Mean Value", "Std Dev", "Text Label"]]

    def generate_df(self, selected_variant: str, round_to: int = 3) -> pd.DataFrame:
        self.build_corpora(selected_variant)
        rows = []
        for ds_lbl, content in self.corpora.items():
            for split_key, _, split_df in self.iter_splits(ds_lbl, only_test=True):
                out = self._summarise(split_df, self.COL, round_to)
                if not out.empty:
                    out["Language"] = ds_lbl
                    out["Split"]   = split_key
                    rows.append(out)
        return (pd.concat(rows, ignore_index=True)
                if rows else
                pd.DataFrame(columns=["Language","Split","Tag","Mean Value","Std Dev","Text Label"]))


In [6]:
helper = LossHelper(dash_data)
df = helper.generate_df("combined")
df

,Tag,Mean Value,Std Dev,Text Label,Language,Split
0,B-LOC,0.294,1.402,0.294 <br>±<br> 1.402,Arabic,Test
1,B-MISC,1.604,2.687,1.604 <br>±<br> 2.687,Arabic,Test
2,B-ORG,1.238,2.611,1.238 <br>±<br> 2.611,Arabic,Test
3,B-PER,0.660,1.956,0.660 <br>±<br> 1.956,Arabic,Test
4,I-LOC,1.152,2.812,1.152 <br>±<br> 2.812,Arabic,Test
5,I-MISC,2.983,3.287,2.983 <br>±<br> 3.287,Arabic,Test
6,I-ORG,1.668,3.275,1.668 <br>±<br> 3.275,Arabic,Test
7,I-PER,0.641,2.102,0.641 <br>±<br> 2.102,Arabic,Test
8,O,0.023,0.298,0.023 <br>±<br> 0.298,Arabic,Test
9,B-LOC,0.400,1.712,0.400 <br>±<br> 1.712,English,Test


In [9]:
helper.corpora['Arabic']['splits']['Test'].columns

Index(['Sentence Ids', 'Token Positions', 'Words', 'Tokens', 'Word Pieces',
       'Core Tokens', 'True Labels', 'Token Selector Id', 'Pred Labels',
       'Agreements', 'X', 'Y', 'Labels', 'Loss Values', 'Token Ids',
       'Global Id', 'True Silhouette', 'Pred Silhouette', 'K=3',
       'Boundary Clusters', 'K=4', 'Entity Clusters', 'K=9', 'Token Clusters',
       'Consistency Count', 'Inconsistency Count', 'Total Train Occurrences',
       'Local Token Entropy', 'Token Max Entropy', 'Dataset Token Entropy',
       'Local Word Entropy', 'Word Max Entropy', 'Dataset Word Entropy',
       'Tokenization Rate', 'Error Type', 'O Confidence', 'B-PER Confidence',
       'I-PER Confidence', 'B-ORG Confidence', 'I-ORG Confidence',
       'B-LOC Confidence', 'I-LOC Confidence', 'B-MISC Confidence',
       'I-MISC Confidence', 'Prediction Entropy', 'Prediction Max Entropy',
       'Token Confidence', 'Variability', 'Pre X', 'Pre Y',
       'Strict True Entities', 'Strict Pred Entities', 'True E

In [46]:
from dash import html, dcc
import pandas as pd
import plotly.express as px

def plot_token_behaviour_bar(
    df: pd.DataFrame,
    *,
    title: str,
    # Column names (auto-detected if None)
    tag_col: str | None = None,         # e.g. "Tag" / "True Labels" / "Entity"
    metric_col: str | None = None,      # e.g. "Mean Value" / "Tokenization Rate"
    level_col: str | None = None,       # e.g. "Level" (Token/Word OR Consistency/Inconsistency)
    text_col: str | None = None,        # e.g. "Text Label"
    color_col: str | None = None,       # e.g. "Dataset" (optional)
    split_col: str | None = None,       # e.g. "Split" (optional)
    # Behaviour flags
    analysis_hint: str | None = None,   # "tokenization" to force NO facets
    facet_by_level: bool = True,        # facet rows by Level if Level column exists
    facet_by_split: bool = False,       # OPTIONAL facet-by-column; default OFF as requested
    # Visual tuning
    tag_order: list[str] | None = None,
    level_order: list[str] | None = None,
    decimals: int = 3,
    y_axis_label: str = "Average Score",
    height: int | None = None,
# ) -> html.Div:
):
    """
    One bar plotter for:
      - Tokenisation Rate: x=Tag, y=metric, color=color_col, no facets.
      - Ambiguity/Consistency: x=Tag, y=Mean, color=Level (legend), facet by Level (rows)
        and optionally facet by Split (columns) when facet_by_split=True.

    All y-axes shown as decimals (no % formatting).
    """

    if df is None or df.empty:
        return html.Div([html.Div("No data to plot.", className="text-muted")])

    data = df.copy()
    cols = set(data.columns)

    def _pick(cands, fb=None):
        for c in cands:
            if c in cols:
                return c
        return fb

    # ---- Auto-detect columns if not provided ----
    tag_col    = tag_col    or _pick(["Tag", "True Labels", "Entity"])
    metric_col = metric_col or _pick(["Mean Value", "Tokenization Rate", "Tokenisation Rate", "value", "score"])
    level_col  = level_col  or _pick(["Level"])
    text_col   = text_col   or _pick(["Text Label"])
    color_col  = color_col  or _pick(["Dataset", "Language"])
    split_col  = split_col  or _pick(["Split"])

    if not tag_col or not metric_col:
        return html.Div([html.Div("Required columns not found (Tag/Metric).", className="text-muted")])

    # ---- Ordering ----
    if tag_order and tag_col in data.columns:
        cats = [t for t in tag_order if t in data[tag_col].astype(str).unique().tolist()]
        if cats:
            data[tag_col] = pd.Categorical(data[tag_col].astype(str), categories=cats, ordered=True)

    if level_order and level_col and level_col in data.columns:
        levs = [l for l in level_order if l in data[level_col].astype(str).unique().tolist()]
        if levs:
            data[level_col] = pd.Categorical(data[level_col].astype(str), categories=levs, ordered=True)

    # ---- Faceting logic ----
    is_tokenization = (analysis_hint == "tokenization") or (not level_col or level_col not in data.columns)
    if is_tokenization:
        facet_row = None
        facet_col = None
        color     = (color_col if color_col in data.columns else None)
    else:
        facet_row = (level_col if facet_by_level and (level_col in data.columns) else None)
        facet_col = (split_col if facet_by_split and split_col and (split_col in data.columns) else None)
    
     # Always color by Dataset/Language if available
    color = (color_col if color_col in data.columns else None)

    # ---- Build figure ----
    fig = px.bar(
        data,
        x=tag_col,
        y=metric_col,
        color=color,
        barmode="group",
        text=(text_col if text_col in data.columns else None),
        facet_row=facet_row,
        facet_col=facet_col,
        category_orders={
            tag_col:   (list(data[tag_col].cat.categories) if hasattr(data.get(tag_col), "cat") else None),
            level_col: (list(data[level_col].cat.categories) if (level_col and hasattr(data.get(level_col), "cat")) else None),
        },
        labels={tag_col: "Entity Tag", metric_col: y_axis_label},
        title=title,
    )

    # ---- Text/axis formatting (decimals only; no %) ----
    if text_col and text_col in data.columns:
        fig.update_traces(textposition="auto", insidetextanchor="middle", cliponaxis=False)
    else:
        fig.update_traces(
            texttemplate=f"%{{y:.{decimals}f}}",
            textposition="auto",          # <--- put labels inside
            insidetextanchor="middle",      # center vertically
            cliponaxis=False
        )


    fig.update_yaxes(tickformat=f".{decimals}f", rangemode="tozero", title=y_axis_label)

    # ---- Layout polish ----
    fig.update_layout(
        template="plotly_white",
        margin=dict(t=60, l=20, r=20, b=40),
        legend_title=None,
        bargap=0.25,
        uniformtext_minsize=10,
        uniformtext_mode="hide",
        height=(height if height is not None else (700 if facet_row or facet_col else 600)),
    )
    fig.update_xaxes(tickangle=-30)

    # Independent ranges for faceted rows/cols
    if facet_row or facet_col:
        fig.for_each_yaxis(lambda yaxis: yaxis.update(matches=None, autorange=True))

    # return html.Div([dcc.Graph(figure=fig)])
    return fig


In [47]:
comp = plot_token_behaviour_bar(
    df,
    title="Consistency vs Inconsistency by Tag (Mean ± SD)",
    level_order=["Consistency Ratio", "Inconsistency Ratio"],
    tag_order=_DEFAULT_TAGS + ["O"],
    facet_by_level=True,
    facet_by_split=False,  # opt-in only if you ever want columns by Split
    y_axis_label="Average Score",
)
comp

In [3]:
import pandas as pd
import plotly.express as px

class BaseAnalysis:
    """Base class for analyzing different variable groups across model variants."""

    VARIANT_MAPPING = {
        "ANERCorp_CamelLab_arabertv02": "Arabic",
        "conll2003_bert": "English"
    }

    def __init__(self, dash_data):
        """Initialize the analysis with the provided dataset."""
        self.dash_data = dash_data
        self.analysis_df = self._prepare_data()

    def _prepare_data(self):
        """Prepare and combine all dataset variants into a single DataFrame."""
        analysis_data = []
        for data_name, data_content in self.dash_data.items():
            data = data_content.analysis_data.copy()
            data["Language"] = self.VARIANT_MAPPING.get(data_name, data_name)
            analysis_data.append(data)
        return pd.concat(analysis_data)

    def filter_and_standardize(self, df, variables):
        """Filter out special tokens and standardize entity label names."""
        # df = df[variables + ["Language", "True Labels"]].copy()
        
        # Remove special tokens
        df = df[~df["True Labels"].isin(["[SEP]", "[CLS]", "IGNORED"])].copy()

        # Standardize label names
        df.loc[:, "True Labels"] = df["True Labels"].replace({"B-PERS": "B-PER", "I-PERS": "I-PER"})
        df.loc[:, "Pred Labels"] = df["Pred Labels"].replace({"B-PERS": "B-PER", "I-PERS": "I-PER"})
    
        
        return df

    def compute_summary(self, df, variables, groupby_cols):
        """Compute mean and standard deviation for each variable per group."""
        summary = df.groupby(groupby_cols)[variables].agg(["mean", "std"]).reset_index()
        # Flatten column names
        summary.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in summary.columns]
        return summary

    def format_long_data(self, summary_df, groupby_cols, metric_label):
        """Convert summary DataFrame into long format for visualization."""
        mean_cols = [col for col in summary_df.columns if "mean" in col]
        std_cols = [col.replace("mean", "std") for col in mean_cols]

        # Melt mean values
        mean_long = summary_df.melt(
            id_vars=[col for col in summary_df.columns if col not in mean_cols + std_cols],
            value_vars=mean_cols,
            var_name=metric_label,
            value_name="Mean Value"
        )

        # Melt standard deviation values
        std_long = summary_df.melt(
            id_vars=[col for col in summary_df.columns if col not in mean_cols + std_cols],
            value_vars=std_cols,
            var_name=metric_label,
            value_name="Std Dev"
        )

        # Clean column names
        mean_long[metric_label] = mean_long[metric_label].str.replace(" mean", "")
        std_long[metric_label] = std_long[metric_label].str.replace(" std", "")

        # Merge mean and std DataFrames
        summary_long = mean_long.merge(std_long, on= groupby_cols + [metric_label], how="left")
        

        # Round values for readability
        summary_long["Mean Value"] = summary_long["Mean Value"].round(3)
        summary_long["Std Dev"] = summary_long["Std Dev"].round(3)

        # Modify text labels to include Std Dev
        summary_long["Text Label"] = summary_long.apply(
            lambda row: f"{row['Mean Value']} <br>±<br>{row['Std Dev']}", axis=1
        )

        return summary_long
    
    def plot_high_level(self, summary_long_df, metric_label, title):
        """Generate a bar plot for high-level (language-based) analysis."""
        fig = px.bar(
            summary_long_df,
            x=metric_label,
            y="Mean Value",
            color="Language",
            barmode="group",
            text="Text Label",
            title=title,
            labels={"Mean Value": "Score", metric_label: "Metrics"},
        )

        fig.update_traces(
            textfont=dict(size=12)
        )

        fig.update_layout(
            template="plotly_white",
            showlegend=True,
        )
        
        fig.show()
        
    def plot_confidence(self, summary_long_df, metric_label, title):
        """Generate a bar plot for high-level (language-based) analysis."""
        fig = px.bar(
            summary_long_df,
            x=metric_label,
            y="Mean Value",
            color="Language",
            facet_row="Type",
            barmode="group",
            text="Text Label",
            title=title,
            facet_row_spacing=0.15,
            labels={"Mean Value": "Average Score", metric_label: "Metrics"},
        )

        fig.update_traces(
            textfont=dict(size=12)
        )

        fig.update_layout(
            template="plotly_white",
            showlegend=True,
            height=600,
            margin=dict(t=50, b=50, l=50, r=50),
        )
        
        fig.show()
    
    def plot_token_confidence(self, summary_long_df, metric_label, title):
        """Generate a bar plot for high-level (language-based) analysis."""
        fig = px.bar(
            summary_long_df,
            x="True Labels",
            y="Mean Value",
            color="Language",
            facet_row="Type",
            barmode="group",
            text="Text Label",
            title=title,
            facet_row_spacing=0.15,
            labels={"Mean Value": "Average Score", metric_label: "Metrics"},
        )

        fig.update_traces(
            textfont=dict(size=12)
        )

        fig.update_layout(
            template="plotly_white",
            showlegend=True,
            height=600,
            margin=dict(t=50, b=50, l=50, r=50),
        )
        
        fig.show()

    def plot_entity_level(self, summary_long_df, metric_label, title, height):
        """Generate a bar plot for entity-based (NER tag) analysis."""
        fig = px.bar(
            summary_long_df,
            x="True Labels",
            y="Mean Value",
            color="Language",
            facet_row=metric_label,
            barmode="group",
            text="Text Label",
            title=title,
            labels={"Mean Value": "Average Score", metric_label: "Metrics"},
            height=height,
            facet_row_spacing=0.0001,
        )

        fig.update_traces(
            textfont=dict(size=12)
        )
        

        fig.update_layout(
            template="plotly_white",
            showlegend=True,
            margin=dict(t=60, b=60, l=80, r=80),
        )
        fig.for_each_yaxis(lambda yaxis: yaxis.update(matches=None, autorange=True))  # Independent y-axes
        
        
        fig.show()


In [ ]:
# class ConsistencyAnalysis(BaseAnalysis):
#     """Class for analyzing consistency metrics across model variants."""

#     VARIABLES = {
#         "Consistency Variables": ["Consistency Ratio", "Inconsistency Ratio"],
#         "Consistency Absolute": ["Consistency Count", "Inconsistency Count"],
#         "OOV": ["Dataset Token Entropy", "Normalized Token Entropy"]
#     }

#     def analyze_high_level(self):
#         """Perform high-level consistency analysis across languages."""
#         variables = self.VARIABLES["Consistency Variables"]
#         oov_variables = self.VARIABLES["OOV"]
#         df = self.analysis_df.copy()
#         print(df.columns)
#         df = df[(df[oov_variables] != -1).all(axis=1)]
#         summary = self.compute_summary(df, variables, groupby_cols=["Language"])
#         summary_long = self.format_long_data(summary, ["Language"], "Consistency Metrics")
#         self.plot_high_level(summary_long, "Consistency Metrics", "High-Level Consistency Analysis")

#     def analyze_entity_level(self,  height=800):
#         """Perform entity-level consistency analysis."""
#         variables = self.VARIABLES["Consistency Variables"]
#         oov_variables = self.VARIABLES["OOV"]
#         df = self.filter_and_standardize(self.analysis_df, variables)
#         df = df[(df[oov_variables] != -1).all(axis=1)]
#         summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
#         summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Consistency Metrics")
#         self.plot_entity_level(summary_long, "Consistency Metrics", "Entity-Level Consistency Analysis", height)




class LossAnalysis(BaseAnalysis):
    """Class for analyzing loss metrics across model variants."""

    VARIABLES = {
        "Loss Metrics": ["Losses"]
    }

    def analyze_high_level(self):
        """Perform high-level loss analysis across languages."""
        variables = self.VARIABLES["Loss Metrics"]
        df = self.analysis_df.copy()
        
        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Loss Metrics")
        self.plot_high_level(summary_long, "Loss Metrics", "High-Level Loss Analysis")

    def analyze_entity_level(self, height=500):
        """Perform entity-level loss analysis."""
        variables = self.VARIABLES["Loss Metrics"]
        df = self.filter_and_standardize(self.analysis_df, variables)

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Loss Metrics")
        self.plot_entity_level(summary_long, "Loss Metrics", "Loss Values Per Entity Tag ", height)
        


class TokenizationAnalysis(BaseAnalysis):
    """Class for analyzing tokenization rates across model variants."""

    VARIABLES = {
        "Tokenization Metrics": ["Tokenization Rate"]
    }

    def analyze_high_level(self):
        """Perform high-level tokenization analysis across languages."""
        variables = self.VARIABLES["Tokenization Metrics"]
        df = self.analysis_df.copy()

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Tokenization Metrics")
        self.plot_high_level(summary_long, "Tokenization Metrics", "High-Level Tokenization Analysis")

    def analyze_entity_level(self, height=500):
        """Perform entity-level tokenization analysis."""
        variables = self.VARIABLES["Tokenization Metrics"]
        df = self.filter_and_standardize(self.analysis_df, variables)

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Tokenization Metrics")
        self.plot_entity_level(summary_long, "Tokenization Metrics", "Tokenisation Rate Per Entity Tag", height)
        

class ConfidenceAnalysis(BaseAnalysis):
    """Class for analyzing confidence metrics across model variants."""

    VARIABLES = {
        "High-Level Confidence": [
            'O Confidence', 
            'B-LOC Confidence', 'I-LOC Confidence',
            'B-PER Confidence', 'I-PER Confidence', 
            'B-ORG Confidence', 'I-ORG Confidence',
            'B-MISC Confidence', 'I-MISC Confidence'
        ],
        "Entity-Level Confidence": ['Token Confidence']
    }

    def analyze_high_level(self):
        """Perform high-level confidence analysis across languages."""
        variables = self.VARIABLES["High-Level Confidence"]
        df = self.analysis_df.copy()

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Confidence Metrics")
        self.plot_high_level(summary_long, "Confidence Metrics", "High-Level Confidence Analysis")
        
    def analyze_high_level_error(self):
        """Perform high-level confidence analysis across languages."""
        variables = self.VARIABLES["High-Level Confidence"]
        df = self.analysis_df.copy()
        df = df[df['Agreements'] == False]

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Confidence Metrics")
        self.plot_high_level(summary_long, "Confidence Metrics", "High-Level Error Prediction Confidence Analysis")
        
    def analyze_high_level_correct(self):
        """Perform high-level confidence analysis across languages."""
        variables = self.VARIABLES["High-Level Confidence"]
        df = self.analysis_df.copy()
        df = df[df['Agreements'] == True]

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Confidence Metrics")
        self.plot_high_level(summary_long, "Confidence Metrics", "High-Level Correct Prediction Confidence Analysis")
        

    def analyze_high_level_summary(self):
        """Perform high-level confidence analysis across languages for both errors and correct predictions."""
        variables = self.VARIABLES["High-Level Confidence"]
        df = self.analysis_df.copy()

        # Compute summary for incorrect predictions
        df_errors = df[df['Agreements'] == False]
        df_errors = self.filter_and_standardize(df_errors, variables)
        summary_errors = self.compute_summary(df_errors, variables, groupby_cols=["Language"])
        summary_errors["Type"] = "Error"

        # Compute summary for correct predictions
        df_correct = df[df['Agreements'] == True]
        df_correct = self.filter_and_standardize(df_correct, variables)
        summary_correct = self.compute_summary(df_correct, variables, groupby_cols=["Language"])
        summary_correct["Type"] = "Correct"

        # Merge both summaries
        summary = pd.concat([summary_errors, summary_correct], ignore_index=True)

        # Convert to long format for plotting
        summary_long = self.format_long_data(summary, ["Language", "Type"], "Confidence Metrics")

        # Plot the results in a single bar chart
        self.plot_confidence(summary_long, "Confidence Metrics", "Comparison of Confidence Scores Across Entity Tags in Correct and Incorrect Predictions")


    def analyze_entity_level_error(self, height=500):
        """Perform entity-level confidence analysis (Token Confidence only)."""
        variables = self.VARIABLES["Entity-Level Confidence"]
        df = self.analysis_df.copy()
        df = df[df['Agreements'] == False]
        df = self.filter_and_standardize(df, variables)
        
        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Token Confidence")
        self.plot_entity_level(summary_long, "Token Confidence", "Entity-Level Error Prediction Confidence Analysis", height)


    def analyze_entity_level_correct(self, height=500):
            """Perform entity-level confidence analysis (Token Confidence only)."""
            variables = self.VARIABLES["Entity-Level Confidence"]
            df = self.analysis_df.copy()
            df = df[df['Agreements'] == True]
            df = self.filter_and_standardize(df, variables)
            
            summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
            
            summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Token Confidence")
            self.plot_entity_level(summary_long, "Token Confidence", "Entity-Level Correct Prediction Confidence Analysis", height)
    
    def analyze_entity_level_summary(self, height=500):
        """
        Perform entity-level confidence analysis (Token Confidence only) for both correct and incorrect predictions
        and present them in a single comparison plot.
        
        Args:
            height (int): Height of the plot.
        """
        variables = self.VARIABLES["Entity-Level Confidence"]
        df = self.analysis_df.copy()

        # Compute summary for incorrect predictions
        df_errors = df[df['Agreements'] == False]
        df_errors = self.filter_and_standardize(df_errors, variables)
        summary_errors = self.compute_summary(df_errors, variables, groupby_cols=["Language", "True Labels"])
        summary_errors["Type"] = "Error"

        # Compute summary for correct predictions
        df_correct = df[df['Agreements'] == True]
        df_correct = self.filter_and_standardize(df_correct, variables)
        summary_correct = self.compute_summary(df_correct, variables, groupby_cols=["Language", "True Labels"])
        summary_correct["Type"] = "Correct"

        # Merge both summaries
        summary = pd.concat([summary_errors, summary_correct], ignore_index=True)

        # Convert to long format for plotting
        summary_long = self.format_long_data(summary, ["Language", "True Labels", "Type"], "Token Confidence")

        # Plot the results in a single bar chart
        self.plot_token_confidence(summary_long, "Token Confidence", "Toke Confidence per Entity Tag Across Correct and Incorrect Predictions")

      
    def detailed_entity_level_errors(self):
        """Perform entity-level confidence analysis (Token Confidence only)."""
        variables = self.VARIABLES["Entity-Level Confidence"]
        df = self.analysis_df.copy()
        df = df[df['Agreements'] == False].copy()
        df = self.filter_and_standardize(df, variables + ['Pred Labels'])

        # Group by 'True Labels' and 'Pred Labels' and calculate weighted average
        grouped_df = df.groupby(['Language', 'True Labels', 'Pred Labels'])[['Token Confidence']].apply(
            lambda x: pd.Series({
                'Weighted Token Confidence': (x['Token Confidence'] * x['Token Confidence'].count()).sum() / x['Token Confidence'].count(),
                'Count': x['Token Confidence'].count()  # Add count for reference
            })
        ).reset_index()

        # Pivot to get heatmap format
        heatmap_data = grouped_df.pivot_table(
            index=['Language', 'True Labels'],
            columns='Pred Labels',
            values='Weighted Token Confidence',
            fill_value=0
        )

        heatmap_arabic = heatmap_data.loc["Arabic"]
        heatmap_english = heatmap_data.loc["English"]

        # Get global min and max values for consistent scaling
        zmin, zmax = heatmap_data.min().min(), heatmap_data.max().max()

        # Create subplots
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=["Arabic", "English"],
            shared_yaxes=True,  # Align y-axis labels
            horizontal_spacing=0.15
        )

        # Add Arabic heatmap
        fig.add_trace(
            go.Heatmap(
                z=heatmap_arabic.values,
                x=heatmap_arabic.columns,
                y=heatmap_arabic.index,
                colorscale="RdBu_r",
                text=heatmap_arabic.round(2).astype(str),  # Convert confidence values to text
                texttemplate="%{text}",  # Format text display
                colorbar=dict(title="Confidence"),
                zmin=zmin, zmax=zmax  # Set shared scale
            ),
            row=1, col=1
        )

        # Add English heatmap
        fig.add_trace(
            go.Heatmap(
                z=heatmap_english.values,
                x=heatmap_english.columns,
                y=heatmap_english.index,
                colorscale="RdBu_r",
                text=heatmap_english.round(2).astype(str),
                texttemplate="%{text}",
                colorbar=dict(title="Confidence"),
                zmin=zmin, zmax=zmax  # Set shared scale
            ),
            row=1, col=2
        )

        # Update layout
        fig.update_layout(
            title_text="Total Token Confidence Confusion Heatmap - Arabic vs. English",
            width=1200, height=600,
            template="plotly_white",
            xaxis_title="Predicted Entity",  # Label x-axis
            yaxis_title="True Entity",  # Label y-axis
        )

        # Update x-axis labels to tilt for both subplots
        fig.update_xaxes(tickangle=-45)  # Tilt x-axis labels for readability

        fig.show()


In [12]:
ar = dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data
en = dash_data['conll2003_bert'].analysis_data

In [13]:
ar.columns

Index(['Sentence Ids', 'Token Positions', 'Words', 'Tokens', 'Word Pieces',
       'Core Tokens', 'True Labels', 'Token Selector Id', 'Pred Labels',
       'Agreements', 'X', 'Y', 'Labels', 'Loss Values', 'Token Ids',
       'Global Id', 'True Silhouette', 'Pred Silhouette', 'K=3',
       'Boundary Clusters', 'K=4', 'Entity Clusters', 'K=9', 'Token Clusters',
       'Consistency Count', 'Inconsistency Count', 'Total Train Occurrences',
       'Local Token Entropy', 'Token Max Entropy', 'Dataset Token Entropy',
       'Local Word Entropy', 'Word Max Entropy', 'Dataset Word Entropy',
       'Tokenization Rate', 'Error Type', 'O Confidence', 'B-PER Confidence',
       'I-PER Confidence', 'B-ORG Confidence', 'I-ORG Confidence',
       'B-LOC Confidence', 'I-LOC Confidence', 'B-MISC Confidence',
       'I-MISC Confidence', 'Prediction Entropy', 'Prediction Max Entropy',
       'Token Confidence', 'Variability', 'Pre X', 'Pre Y',
       'Strict True Entities', 'Strict Pred Entities', 'True E

In [14]:
ar_e = ar[(ar['Agreements'] ==False)].groupby(['True Labels', 'Pred Labels'])['Token Confidence'].mean().reset_index()

ar_e



,True Labels,Pred Labels,Token Confidence
0,B-LOC,B-MISC,0.634367
1,B-LOC,B-ORG,0.842200
2,B-LOC,B-PER,0.929100
3,B-LOC,I-LOC,0.272400
4,B-LOC,I-ORG,0.843050
5,B-LOC,I-PER,0.994700
6,B-LOC,O,0.897625
7,B-MISC,B-LOC,0.741125
8,B-MISC,B-ORG,0.798230
9,B-MISC,I-MISC,0.583300


In [15]:
# Filter the dataframe
ar_e = ar[ar['Agreements'] == False].copy()
ar_e['Count'] = ar_e.groupby(['True Labels', 'Pred Labels'])['Token Confidence'].transform('count')
# Group by 'True Labels' and 'Pred Labels' and calculate weighted average
weighted_ar_e = ar_e.groupby(['True Labels', 'Pred Labels']).apply(
    lambda x: pd.Series({
        'Weighted Token Confidence': (x['Token Confidence'] * x['Token Confidence'].count()).sum() / x['Token Confidence'].count(),
        'Count': x['Token Confidence'].count()  # Add count for reference
    })
).reset_index()

heatmap_data = weighted_ar_e.pivot_table(
            index=['True Labels'], 
            columns='Pred Labels', 
            values='Weighted Token Confidence', 
            fill_value=0
        )
weighted_ar_e.tail()



/var/folders/v3/x7sg5lfs7f97bw4zryqy1w1c0000gn/T/ipykernel_67558/2269311503.py:5: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,True Labels,Pred Labels,Weighted Token Confidence,Count
52,O,B-PER,30.2873,34.0
53,O,I-LOC,1.9681,2.0
54,O,I-MISC,7.1308,10.0
55,O,I-ORG,22.3054,25.0
56,O,I-PER,5.5636,6.0


In [16]:
((0.9789*2)+(0.9892*2))/2



1.9681

In [17]:
print(ar[(ar['Agreements'] ==  False)
         &(ar['True Labels'] ==  'I-MISC')
         &(ar['Pred Labels'] ==  'O')
        ][['True Labels', 'Pred Labels', 'Token Confidence']].describe())

       Token Confidence
count         59.000000
mean           0.870208
std            0.178122
min            0.461000
25%            0.706550
50%            0.989000
75%            0.999400
max            1.000000


In [18]:
print(ar[(ar['Agreements'] ==  False)
         &(ar['True Labels'] ==  'I-LOC')
         &(ar['Pred Labels'] ==  'O')
        ][['True Labels', 'Pred Labels', 'Token Confidence', 'Losses']])

KeyError: "['Losses'] not in index"

# Analysis

### ambiguity

In [ ]:

class AmbiguityAnalysis(BaseAnalysis):
    """Class for analyzing ambiguity-related metrics."""

    VARIABLES = {
        "Token Level": ["Token Ambiguity"],
        "Word Level": ["Word Ambiguity"]
    }

    def analyze_token_high_level(self):
        """Perform high-level token ambiguity analysis across languages."""
        variables = self.VARIABLES["Token Level"]
        df = self.analysis_df.copy()
        
        # Remove OOV entropy (-1) specifically for token-level metrics
        df = df[(df[variables] != -1).all(axis=1)]

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Ambiguity Metrics")
        self.plot_high_level(summary_long, "Ambiguity Metrics", "High-Level Token Ambiguity Analysis")

    def analyze_word_high_level(self):
        """Perform high-level word ambiguity analysis across languages."""
        variables = self.VARIABLES["Word Level"]
        df = self.analysis_df.copy()
        
        # Remove OOV entropy (-1) specifically for word-level metrics
        df = df[(df[variables] != -1).all(axis=1)]

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Ambiguity Metrics")
        self.plot_high_level(summary_long, "Ambiguity Metrics", "High-Level Word Ambiguity Analysis")

    def analyze_token_entity_level(self, height=800):
        """Perform entity-level token ambiguity analysis."""
        variables = self.VARIABLES["Token Level"]
        df = self.filter_and_standardize(self.analysis_df, variables)

        # Remove OOV entropy (-1) specifically for token-level metrics
        df = df[(df[variables] != -1).all(axis=1)]

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Ambiguity Metrics")
        self.plot_entity_level(summary_long, "Ambiguity Metrics", "Entity-Level Token Ambiguity Analysis", height)

    def analyze_word_entity_level(self, height=800):
        """Perform entity-level word ambiguity analysis."""
        variables = self.VARIABLES["Word Level"]
        df = self.filter_and_standardize(self.analysis_df, variables)

        # Remove OOV entropy (-1) specifically for word-level metrics
        df = df[(df[variables] != -1).all(axis=1)]

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Ambiguity Metrics")
        self.plot_entity_level(summary_long, "Ambiguity Metrics", "Entity-Level Word Ambiguity Analysis", height)
    def analyze_merged_entity_level(self, height=800):
        """Perform combined entity-level ambiguity analysis for token and word level."""
        token_vars = self.VARIABLES["Token Level"]
        word_vars = self.VARIABLES["Word Level"]

        # Prepare data for both levels
        df_token = self.filter_and_standardize(self.analysis_df.copy(), token_vars)
        df_word = self.filter_and_standardize(self.analysis_df.copy(), word_vars)

        # Remove OOV values
        df_token = df_token[(df_token[token_vars] != -1).all(axis=1)]
        df_word = df_word[(df_word[word_vars] != -1).all(axis=1)]

        # Compute summaries
        token_summary = self.compute_summary(df_token, token_vars, groupby_cols=["Language", "True Labels"])
        word_summary = self.compute_summary(df_word, word_vars, groupby_cols=["Language", "True Labels"])

        # Add a 'Level' column to distinguish them
        token_summary["Level"] = "Token Level"
        word_summary["Level"] = "Word Level"

        # Rename ambiguity column for merging
        token_summary = token_summary.rename(columns={token_vars[0]: "Ambiguity"})
        word_summary = word_summary.rename(columns={word_vars[0]: "Ambiguity"})

        # Merge both summaries
        merged_summary = pd.concat([word_summary, token_summary], ignore_index=True)
        # return merged_summary
        # Format for plotting
        summary_long = self.format_long_data(merged_summary, ["Language", "True Labels", "Level"], "Ambiguity Metrics")

        # Plot
        self.plot_entity_level(summary_long, "Entity-Tag Ambiguity Comparison (Token vs Word)")
        # return self.plot_entity_level(summary_long, "Ambiguity Metrics", "Entity-Level Ambiguity Comparison (Token vs Word)", height, y_col="Mean Value")

    def plot_entity_level(self, df, title, height=800):
        import plotly.express as px
        

        fig = px.bar(
            df,
            x="True Labels",
            y="Mean Value",
            color="Language",
            barmode="group",
            facet_row="Level",
            text='Text Label',
            title=title
        )
        # fig.update_traces(texttemplate='%{y:.3f}', textposition='outside')
        fig.update_layout(uniformtext_minsize=8, template="plotly_white", height=800,
            showlegend=True, 
            uniformtext_mode='hide')
        fig.show()


In [ ]:

ambiguity_analysis = AmbiguityAnalysis(dash_data)
ambiguity_analysis.analyze_merged_entity_level()


In [ ]:
# ambiguity_analysis = AmbiguityAnalysis(dash_data)
# ambiguity_analysis.analyze_token_high_level()  # High-level ambiguity analysis
# ambiguity_analysis.analyze_word_high_level()  # High-level ambiguity analysis
# ambiguity_analysis.analyze_token_entity_level()  # Entity-level ambiguity analysis
# ambiguity_analysis.analyze_word_entity_level()  # Entity-level ambiguity analysis


### Tokenisation

In [ ]:
variable_analysis = TokenizationAnalysis(dash_data)
# variable_analysis.analyze_high_level()  # High-level ambiguity analysis
variable_analysis.analyze_entity_level()  # Entity-level ambiguity analysis


In [20]:
helper.corpora.keys()

NameError: name 'helper' is not defined

### consistency

In [19]:


class ConsistencyAnalysis(BaseAnalysis):
    """Class for analyzing consistency metrics across model variants."""

    VARIABLES = {
        "Consistency Variables": [ "Inconsistency Ratio"],
        "Consistency Absolute": ["Consistency Count", "Inconsistency Count"],
        "OOV": ["Token Ambiguity"]
    }

    def analyze_high_level(self):
        """Perform high-level consistency analysis across languages (Graph)."""
        variables = self.VARIABLES["Consistency Variables"]
        oov_variables = self.VARIABLES["OOV"]
        df = self.analysis_df.copy()
        df = df[(df[oov_variables] != -1).all(axis=1)]
        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Consistency Metrics")
        self.plot_high_level(summary_long, "Consistency Metrics", "High-Level Consistency Analysis")

    def analyze_entity_level(self, height=800):
        """Perform entity-level consistency analysis (Graph)."""
        variables = self.VARIABLES["Consistency Variables"]
        oov_variables = self.VARIABLES["OOV"]
        df = self.analysis_df.copy()
        df = df[(df[oov_variables] != -1).all(axis=1)]
        df = self.filter_and_standardize(df, variables)
        
        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Consistency Metrics")
        self.plot_entity_level(summary_long, "Consistency Metrics", "Entity-Tag Consistency / In Consistency Analysis", height)

    def generate_high_level_table(self):
        """Generate a table for high-level consistency analysis."""
        variables = self.VARIABLES["Consistency Absolute"]
        oov_variables = self.VARIABLES["OOV"]
        df = self.analysis_df.copy()
        df = df[(df[oov_variables] != -1).all(axis=1)]
        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        
        
        table = pd.DataFrame(summary)
        table = table.round(2)
        # table.rename(columns={"Language": "Dataset", 
        #                       "Consistency Count": "Consistent Tokens", 
        #                       "Inconsistency Count": "Inconsistent Tokens"}, inplace=True)
        
        display(table)
    def generate_entity_level_table(self):
        """Generate a table for entity-level consistency analysis."""
        variables = self.VARIABLES["Consistency Absolute"]
        oov_variables = self.VARIABLES["OOV"]
        df = self.analysis_df.copy()
        df = df[(df[oov_variables] != -1).all(axis=1)]
        df = self.filter_and_standardize(self.analysis_df, variables)
        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])

        
        table = pd.DataFrame(summary)
        table = table.round(2)
        # table.rename(columns={"Language": "Dataset", 
        #                       "True Labels": "Entity Tag",
        #                       "Consistency Count": "Consistent Tokens", 
        #                       "Inconsistency Count": "Inconsistent Tokens"}, inplace=True)
        display(table)

In [20]:

variable_analysis = ConsistencyAnalysis(dash_data)
# variable_analysis.analyze_high_level()
variable_analysis.generate_high_level_table()
variable_analysis.analyze_entity_level()
variable_analysis.generate_entity_level_table()


,Language,Consistency Count mean,Consistency Count std,Inconsistency Count mean,Inconsistency Count std
0,Arabic,566.12,1190.27,5.00,73.08
1,English,1378.19,2321.67,33.35,251.22


,Language,True Labels,Consistency Count mean,Consistency Count std,Inconsistency Count mean,Inconsistency Count std
0,Arabic,B-LOC,24.30,36.15,2.24,8.15
1,Arabic,B-MISC,14.76,31.05,8.12,25.27
2,Arabic,B-ORG,10.53,18.62,24.56,85.03
3,Arabic,B-PER,5.31,13.44,15.94,98.13
4,Arabic,I-LOC,44.71,44.83,93.46,87.98
5,Arabic,I-MISC,1.22,3.62,152.79,675.61
6,Arabic,I-ORG,13.72,38.57,42.12,284.18
7,Arabic,I-PER,8.62,22.96,16.27,78.69
8,Arabic,O,583.13,1206.33,1.29,4.77
9,English,B-LOC,53.56,63.43,48.54,92.30


### Loss

In [109]:
loss_analysis = LossAnalysis(dash_data)
# loss_analysis.analyze_high_level()
loss_analysis.analyze_entity_level()


### Confidence

In [119]:
confidence_analysis = ConfidenceAnalysis(dash_data)
confidence_analysis.analyze_high_level_summary()
confidence_analysis.analyze_entity_level_summary()
# confidence_analysis.analyze_high_level_error()
# confidence_analysis.analyze_high_level_correct()
# confidence_analysis.analyze_entity_level_error()
# confidence_analysis.analyze_entity_level_correct()
confidence_analysis.detailed_entity_level_errors()


### Prediction Entropy

In [15]:
class PredictionAnalysis(BaseAnalysis):
    """Class for analyzing prediction variability across model variants."""

    VARIABLES = {
        "Prediction Metrics": ["Prediction Uncertainty"]
    }

    def analyze_high_level(self):
        """Perform high-level prediction variability analysis across languages."""
        variables = self.VARIABLES["Prediction Metrics"]
        df = self.analysis_df.copy()

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Prediction Metrics")
        self.plot_high_level(summary_long, "Prediction Metrics", "High-Level Prediction Distribution Analysis")

    def analyze_entity_level_correct(self, height=800):
        """Perform entity-level prediction variability analysis."""
        variables = self.VARIABLES["Prediction Metrics"]
        df = self.analysis_df.copy()
        df = df[df['Agreements'] == True]
        df = self.filter_and_standardize(df, variables)

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Prediction Metrics")
        self.plot_entity_level(summary_long, "Prediction Metrics", "Entity-Level Correct Prediction Distribution Analysis", height)
    
    def analyze_entity_level_error(self, height=800):
        """Perform entity-level prediction variability analysis."""
        variables = self.VARIABLES["Prediction Metrics"]
        df = self.analysis_df.copy()
        df = df[df['Agreements'] == False]
        df = self.filter_and_standardize(df, variables)

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Prediction Metrics")
        self.plot_entity_level(summary_long, "Prediction Metrics", "Entity-Level Incorrect Prediction Distribution Analysis", height)
        
    
    def analyze_entity_level_merged(self, height=800):
        """Perform merged entity-level prediction variability analysis (Correct vs Incorrect)."""
        variables = self.VARIABLES["Prediction Metrics"]
        df_error = self.analysis_df.copy()
        df_error = df_error[df_error["Agreements"] == False]
        df_error = self.filter_and_standardize(df_error, variables)
        df_error["Type"] = "Error"
        
        
        # Create two filtered versions of the data
        df_correct = self.analysis_df.copy()
        df_correct = df_correct[df_correct["Agreements"] == True]
        df_correct = self.filter_and_standardize(df_correct, variables)
        df_correct["Type"] = "Correct"

        

        # Combine them
        df_combined = pd.concat([df_error, df_correct], ignore_index=True)

        # Compute summary stats
        summary = self.compute_summary(df_combined, variables, groupby_cols=["Language", "True Labels", "Type"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels", "Type"], "Prediction Metrics")

        # Plot
        self.plot_entity_level(summary_long,  "Entity-Tag Prediction Uncertainty (Correct vs Incorrect)", height)

    def plot_entity_level(self, df, title, height=800):
        import plotly.express as px

        fig = px.bar(
            df,
            x="True Labels",
            y="Mean Value",
            color="Language",
            barmode="group",
            facet_row="Type",  # new facet dimension
            text='Text Label',
            title=title,
            height=height,
            template="plotly_white"
        )
        fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide', showlegend=True)
        fig.show()




In [16]:
variable_analysis = PredictionAnalysis(dash_data)
variable_analysis.analyze_entity_level_merged()


In [91]:
# variable_analysis = PredictionAnalysis(dash_data)
# variable_analysis.analyze_high_level()
# variable_analysis.analyze_entity_level_correct()
# variable_analysis.analyze_entity_level_error()


In [ ]:
ar[(ar['Agreements'] ==  False)
         &(ar['True Labels'] ==  'I-MISC')
        #  &(ar['Pred Labels'] ==  'O')
        ][
            # confidence_variables + 
          ['True Labels', 'Pred Labels', 'Token Confidence', 'Losses', 'Prediction Max Entropy', 'Prediction Entropy', 'Normalized Prediction Entropy', 'Variability']].describe()


In [ ]:
en[(en['Agreements'] ==  False)
         &(en['True Labels'] ==  'I-MISC')
        #  &(ar['Pred Labels'] ==  'O')
        ][
            # confidence_variables +
            ['True Labels', 'Pred Labels', 'Token Confidence', 'Losses', 'Prediction Max Entropy', 'Prediction Entropy', 'Normalized Prediction Entropy', 'Variability']].describe()


### Silhouette Score

In [112]:


class SilhouetteAnalysis(BaseAnalysis):
    """Class for analyzing silhouette scores across model variants."""

    VARIABLES = {
        "Silhouette Scores": ["True Silhouette", "Pred Silhouette"]
    }

    def analyze_high_level(self):
        """Perform high-level silhouette score analysis across languages."""
        variables = self.VARIABLES["Silhouette Scores"]
        df = self.analysis_df.copy()
        print(df.columns)

        summary = self.compute_summary(df, variables, groupby_cols=["Language"])
        summary_long = self.format_long_data(summary, ["Language"], "Silhouette Scores")
        self.plot_high_level(summary_long, "Silhouette Scores", "High-Level Silhouette Score Analysis")

    def analyze_entity_level(self, height=800):
        """Perform entity-level silhouette score analysis."""
        variables = self.VARIABLES["Silhouette Scores"]
        df = self.filter_and_standardize(self.analysis_df, variables)

        summary = self.compute_summary(df, variables, groupby_cols=["Language", "True Labels"])
        summary_long = self.format_long_data(summary, ["Language", "True Labels"], "Silhouette Scores")
        self.plot_entity_level(summary_long, "Silhouette Scores", "Entity-Tag Silhouette Score Analysis", height)
    
    

In [113]:
silhouette_analysis = SilhouetteAnalysis(dash_data)
# silhouette_analysis.analyze_high_level()
silhouette_analysis.analyze_entity_level()


# Investigation

In [ ]:
class JointAnalysis:
    """Class to perform joint analysis on model variants."""

    # Define mapping for variant names
    VARIANT_MAPPING = {
        "ANERCorp_CamelLab_arabertv02": "Arabic",
        "conll2003_bert": "English"
    }

    def __init__(self, dash_data):
        """
        Initialize the analysis.

        Parameters:
        - dash_data (dict): Dictionary containing dataset variants.
        """
        self.dash_data = dash_data
        self.analysis_df = self._prepare_data()

    def _prepare_data(self):
        """Prepare and combine all dataset variants into a single DataFrame."""
        analysis_data = []
        for data_name, data_content in self.dash_data.items():
            data = data_content.analysis_data.copy()
            data["Language"] = self.VARIANT_MAPPING.get(data_name, data_name)  # Map variant names
            analysis_data.append(data)
        return pd.concat(analysis_data)

    def compute_summary(self, data, variables):
        """Compute mean and standard deviation for each variable per model variant."""
        summary = data[variables + ["Language"]].groupby("Language").agg(["mean", "std"]).reset_index()

        # Flatten column names
        summary.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in summary.columns]
        return summary

    def format_long_data(self, summary_df, metric_label):
        """Convert summary DataFrame into long format for visualization."""
        mean_cols = [col for col in summary_df.columns if "mean" in col]
        std_cols = [col.replace("mean", "std") for col in mean_cols]

        # Melt mean values
        mean_long = summary_df.melt(
            id_vars=["Language"], 
            value_vars=mean_cols, 
            var_name=metric_label, 
            value_name="Mean Value"
        )

        # Melt standard deviation values
        std_long = summary_df.melt(
            id_vars=["Language"], 
            value_vars=std_cols, 
            var_name=metric_label, 
            value_name="Std Dev"
        )

        # Clean column names (remove "mean"/"std" suffix from metric names)
        mean_long[metric_label] = mean_long[metric_label].str.replace(" mean", "")
        std_long[metric_label] = std_long[metric_label].str.replace(" std", "")

        # Merge mean and std DataFrames
        summary_long = mean_long.merge(
            std_long, 
            on=["Language", metric_label], 
            how="left"
        )

        # Round values for better readability
        summary_long["Mean Value"] = summary_long["Mean Value"].round(2)
        summary_long["Std Dev"] = summary_long["Std Dev"].round(2)

        # Create text label to display above error bars (mean ± std)
        summary_long["Text Label"] = summary_long.apply(
            lambda row: f"{row['Mean Value']}<br>±<br>{row['Std Dev']}", axis=1
        )

        return summary_long

    def plot_summary(self, summary_long_df, metric_label, title):
        """Generate a bar plot with error bars representing standard deviation."""
        fig = px.bar(
            summary_long_df,
            x=metric_label,
            y="Mean Value",
            color="Language",
            barmode="group",
            text="Text Label",  # Show mean values inside bars
            title=title,
            labels={"Mean Value": "Score", metric_label: "Metrics"},
            height=800,
            width=1000
        )

        # Add standard deviation as error bars
        fig.update_traces(
            error_y=dict(
                type="data",
                array=summary_long_df["Text Label"],
                width=2,
                thickness=0.5,
                visible=True
            ),
            # textposition="outside",
            textfont=dict(size=12)
        )

      
     
        # Adjust layout for readability
        fig.update_layout(
            template="plotly_white",
            showlegend=True,
            # xaxis_tickangle=-45
        )
        
        fig.show()

    def analyze(self, variables, metric_label="Metric", title="Comparison of Variables Across Model Variants"):
        """Run the full analysis pipeline."""
        summary = self.compute_summary(variables)
        summary_long = self.format_long_data(summary, metric_label)
        self.plot_summary(summary_long, metric_label, title)
        
    def analyse_group(self):
        
        # Define variable groups
        variable_groups = {
            "Consistency Metrics": ["Consistency Ratio", "Inconsistency Ratio"],
            "Ambiguity Metrics": [
                "Dataset Token Entropy", "Dataset Word Entropy",
                "Normalized Token Entropy", "Normalized Word Entropy"
            ],
            "Tokenization Metrics": ['Tokenization Rate'],
            "Loss Metrics": ['Losses'],
            "Confidence Metrics": [
                'O Confidence', 'B-LOC Confidence', 'I-LOC Confidence',
                'B-PER Confidence', 'I-PER Confidence', 
                'B-ORG Confidence', 'I-ORG Confidence',
                'B-MISC Confidence', 'I-MISC Confidence'
            ],
            "Prediction Metrics": [
                'Variability', 'Normalized Prediction Entropy'
            ],
            "Silhouette Scores": ['True Silhouette Score', 'Pred Silhouette Score']
        }

        # Loop through each variable category
        for group_name, variables in variable_groups.items():
            # Check if any of the requested variables exist in the dataset
            relevant_vars = [var for var in variables if var in self.analysis_df.columns]
            if not relevant_vars:
                continue  # Skip this category if none of its variables exist
            
            # Handle Ambiguity Metrics separately (Remove -1 from OOV adjustments)
            if group_name == "Ambiguity Metrics":
                temp_df = self.analysis_df.copy()
                for var in relevant_vars:
                    temp_df = temp_df[temp_df[var] != -1] 
            else:
                temp_df = self.analysis_df  # Use the original data for other metrics

            # Temporarily update self.variables for the current category
            self.variables = relevant_vars

            # Compute summary statistics
            summary = self.compute_summary(temp_df, relevant_vars)

            # Format data for visualization
            summary_long = self.format_long_data(summary, group_name)

            # Generate a separate plot for each category
            plot_title = f"Comparison of {group_name} Across Languages"
            self.plot_summary(summary_long, group_name, plot_title)



class EntityJointAnalysis:
    """Class to analyze consistency variables across model variants at the entity level."""

    # Define mapping for variant names
    VARIANT_MAPPING = {
        "ANERCorp_CamelLab_arabertv02": "Arabic",
        "conll2003_bert": "English"
    }

    def __init__(self, dash_data):
        """
        Initialize the entity-level analysis.

        Parameters:
        - dash_data (dict): Dictionary containing dataset variants.
        
        """
        self.dash_data = dash_data
        self.analysis_df = self._prepare_data()

    def _prepare_data(self):
        """Prepare and combine all dataset variants into a single DataFrame."""
        analysis_data = []
        for data_name, data_content in self.dash_data.items():
            data = data_content.analysis_data.copy()
            data["Language"] = self.VARIANT_MAPPING.get(data_name, data_name)  # Map variant names
            analysis_data.append(data)
        return pd.concat(analysis_data)

    def filter_and_standardize(self, variables):
        """Filter out unwanted special tokens and standardize entity label names."""
        df = self.analysis_df[variables + ["Language", "True Labels"]].copy()
        
        # Remove special tokens
        df = df[~df["True Labels"].isin(["[SEP]", "[CLS]", "IGNORED"])]
        
        # Standardize label names
        df["True Labels"] = df["True Labels"].replace({"B-PERS": "B-PER", "I-PERS": "I-PER"})
        
        return df

    def compute_summary(self, df, variables):
        """Compute mean and standard deviation for each variable per model variant and entity label."""
        summary = df.groupby(["Language", "True Labels"])[variables].agg(["mean", "std"]).reset_index()

        # Flatten column names
        summary.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in summary.columns]
        return summary

    def format_long_data(self, summary_df, metric_label):
        """Convert summary DataFrame into long format for visualization."""
        mean_cols = [col for col in summary_df.columns if "mean" in col]
        std_cols = [col.replace("mean", "std") for col in mean_cols]

        # Melt mean values
        mean_long = summary_df.melt(
            id_vars=["Language", "True Labels"],
            value_vars=mean_cols,
            var_name=metric_label,
            value_name="Average Value"
        )

        # Melt standard deviation values
        std_long = summary_df.melt(
            id_vars=["Language", "True Labels"],
            value_vars=std_cols,
            var_name=metric_label,
            value_name="Std Dev"
        )

        # Remove "mean" and "std" suffixes in "Consistency Metric" names
        mean_long[metric_label] = mean_long[metric_label].str.replace(" mean", "")
        std_long[metric_label] = std_long[metric_label].str.replace(" std", "")

        # Merge mean and std DataFrames
        summary_long = mean_long.merge(
            std_long, 
            on=["Language", "True Labels", metric_label], 
            how="left"
        )

        # Round values for better readability
        summary_long["Average Value"] = summary_long["Average Value"].round(3)
        summary_long["Std Dev"] = summary_long["Std Dev"].round(3)

        # Modify text labels to include Std Dev
        summary_long["text_label"] = summary_long.apply(
            lambda row: f"{row['Average Value']} <br>±<br>{row['Std Dev']}", axis=1
        )

        return summary_long

    def plot_summary(self, summary_long_df, metric_label, title):
        """Generate a bar plot comparing entity-based consistency variables."""
        fig = px.bar(
            summary_long_df,
            x="True Labels",
            y="Average Value",
            color="Language",
            facet_row=metric_label,  # Each entity type in a different row
            barmode="group",
            text="text_label",
            title=title,
            labels={"Average Value": "Average Score", metric_label: "Metrics"},
            height=800,
            width=1000
        )

        # Add standard deviation as error bars
        fig.update_traces(
            # textposition="outside",  
            textfont=dict(size=12),
            # insidetextanchor="start",  
        )

        # Adjust layout for readability
        fig.update_layout(
            template="plotly_white",
            showlegend=True,
            margin=dict(t=120, b=80, l=80, r=80),
            xaxis_tickangle=-45
        )
        
        fig.show()

    def analyze(self, variables, metric_label="Metric", title="Comparison of Consistency Variables Across Model Variants and True Labels"):
        """Run the full entity-based analysis pipeline."""
        filtered_df = self.filter_and_standardize(variables)
        summary = self.compute_summary(filtered_df, variables)
        summary_long = self.format_long_data(summary, metric_label)
        self.plot_summary(summary_long, metric_label, title)
        
    def analyze_group(self):
        """
        Generate separate plots for different categories of entity-level variables.
        """
        # Define variable groups
        variable_groups = {
            "Consistency Metrics": ["Consistency Ratio", "Inconsistency Ratio"],
            "Ambiguity Metrics - Dataset Level": [
                "Dataset Token Entropy", "Dataset Word Entropy",
            ],
            "Ambiguity Metrics - Token Level": [
                "Normalized Token Entropy", "Normalized Word Entropy"
            ],
            "Tokenization Metrics": ['Tokenization Rate'],
            "Loss Metrics": ['Losses'],
            # "Confidence Metrics": [
            #     'Token Confidence'
            #     # , 'B-LOC Confidence', 'I-LOC Confidence',
            #     # 'B-PER Confidence', 'I-PER Confidence', 
            #     # 'B-ORG Confidence', 'I-ORG Confidence',
            #     # 'B-MISC Confidence', 'I-MISC Confidence'
            # ],
            "Prediction Metrics": [
                'Variability', 'Normalized Prediction Entropy'
            ],
            "Silhouette Scores": ['True Silhouette Score', 'Pred Silhouette Score']
        }

        # Filter and standardize data
        

        # Loop through each variable category
        for group_name, variables in variable_groups.items():
            # Filter variables that exist in the dataset
            relevant_vars = [var for var in variables if var in self.analysis_df.columns]
            if not relevant_vars:
                continue  # Skip if none of the variables exist

            # Temporarily update self.variables for the current category
            self.variables = relevant_vars
            
            filtered_df = self.filter_and_standardize(relevant_vars)
            if "Ambiguity" in group_name:
                for var in relevant_vars:
                    filtered_df = filtered_df[filtered_df[var] != -1]  # Remove OOV entropy (-1)
                    
            # Compute summary statistics
            summary = self.compute_summary(filtered_df, relevant_vars)

            # Format data for visualization
            summary_long = self.format_long_data(summary, group_name)

            # Generate a separate plot for each category
            plot_title = f"Comparison of {group_name} Across Model Variants and True Labels"
            self.plot_summary(summary_long, group_name, plot_title)



In [ ]:
consistency_variables = ["Consistency Ratio", "Inconsistency Ratio"]

ambiguity_variables = [
    # "Local Token Entropy", "Local Word Entropy",
    "Dataset Token Entropy", "Dataset Word Entropy",
    "Normalized Token Entropy", "Normalized Word Entropy"
]

tokenization_variables = ['Tokenization Rate']
loss_variables = ['Losses']

confidence_variables = [
    'O Confidence', 
    'B-LOC Confidence', 'I-LOC Confidence',
    'B-PER Confidence', 'I-PER Confidence', 
    'B-ORG Confidence', 'I-ORG Confidence',
	'B-MISC Confidence', 'I-MISC Confidence'
]
prediction_variables = [
    # 'Prediction Entropy',
    'Variability', 'Normalized Prediction Entropy'
]

silhouette_variables = ['True Silhouette Score', 'Pred Silhouette Score']

In [ ]:
ar = dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data
en = dash_data['conll2003_bert'].analysis_data

In [ ]:
ar[ar['Dataset Word Entropy'] > 0]['Dataset Word Entropy'].mean()

In [ ]:
en[en['Dataset Word Entropy'] > 0]['Dataset Word Entropy'].mean()

In [ ]:
en[ambiguity_variables]['Dataset Word Entropy'].mean()


In [ ]:
df.columns

In [ ]:
analysis.analysis_df[analysis.analysis_df['Dataset Token Entropy']>0].groupby('Language')['Dataset Token Entropy'].mean()

In [ ]:
analysis.analysis_df[analysis.analysis_df['Dataset Word Entropy']>0].groupby('Language')['Dataset Word Entropy'].mean()

In [ ]:
analysis = JointAnalysis(dash_data)
analysis.analyse_group()

In [ ]:
analysis = EntityJointAnalysis(dash_data)
analysis.analyze_group()